# Module 17: Computer Vision with CNNs

**Estimated Time**: 90 minutes

## Learning Objectives

By the end of this module, you will master computer vision with cnns.

Topics covered:
- Computer Vision Introduction
- Convolutional Neural Networks
- Pooling and Padding
- Building CNN Architecture
- Transfer Learning with Pre-trained Models
- Image Classification Project
- Data Augmentation
- Model Deployment

## Prerequisites

- Modules 00-11 completed
- Intermediate Python and ML knowledge

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

print("Libraries loaded successfully!")

## 1. Computer Vision Introduction

**Computer Vision** (CV) enables machines to understand and interpret visual information from the world.

### What is Computer Vision?

> **"Teaching computers to see and understand images like humans do"**

**Goal**: Extract meaningful information from digital images and videos

**Examples:**
- 📸 **Photo tagging**: Facebook recognizes faces
- 🚗 **Self-driving cars**: Detect pedestrians, traffic signs
- 🏥 **Medical imaging**: Diagnose diseases from X-rays, MRIs
- 📱 **Face unlock**: iPhone Face ID
- 🛒 **Product search**: Google Lens
- 🎨 **Art generation**: DALL-E, Midjourney

### The Challenge: Images are Just Numbers

**Human sees**: Cat, dog, tree, car
**Computer sees**: Matrix of pixel values (0-255)

```
Image (28×28 grayscale) = 784 numbers
Image (224×224 RGB) = 150,528 numbers!
```

**Problem**: How do we learn patterns from millions of pixel values?
**Solution**: Convolutional Neural Networks (CNNs)

### Why Regular Neural Networks Fail for Images

**Example**: 224×224 RGB image = 150,528 input neurons

**Problems:**
1. **Too many parameters**: Hidden layer with 1000 neurons = 150M weights!
2. **No spatial awareness**: Nearby pixels are related, but fully connected networks don't know this
3. **Not translation invariant**: Cat in top-left ≠ Cat in bottom-right
4. **Computationally expensive**: Training is impossibly slow

### CNNs: The Solution

**Key Ideas:**
1. **Local connectivity**: Neurons only connect to small regions
2. **Parameter sharing**: Same filters applied across entire image
3. **Translation invariance**: Detect patterns anywhere in image
4. **Hierarchical features**: Low-level (edges) → High-level (objects)

### How CNNs Learn Features

**Traditional ML**: Hand-craft features (SIFT, HOG, etc.)
**CNNs**: Learn features automatically!

**Layer 1**: Detects edges, colors
**Layer 2**: Detects textures, patterns
**Layer 3**: Detects parts (eyes, wheels, leaves)
**Layer 4**: Detects objects (faces, cars, trees)

### Image Representation

**Grayscale Image:**
- Shape: (height, width)
- Values: 0 (black) to 255 (white)
- Example: MNIST digits (28×28)

**RGB Color Image:**
- Shape: (height, width, 3)
- 3 channels: Red, Green, Blue
- Each pixel: (R, G, B) where each is 0-255
- Example: CIFAR-10 (32×32×3)

### Computer Vision Tasks

| Task | Description | Output |
|------|-------------|--------|
| **Image Classification** | What is in this image? | Single label (cat, dog, car) |
| **Object Detection** | Where are objects? | Bounding boxes + labels |
| **Semantic Segmentation** | Classify each pixel | Pixel-wise labels |
| **Instance Segmentation** | Separate object instances | Individual object masks |
| **Image Generation** | Create new images | Synthetic image |

### Historical Milestones

- **2012**: AlexNet wins ImageNet (CNNs revolution begins)
- **2014**: VGGNet, GoogLeNet (deeper networks)
- **2015**: ResNet (152 layers with skip connections!)
- **2017**: Transformers for vision (ViT)
- **2020s**: Diffusion models (DALL-E, Stable Diffusion)

Let's visualize how computers see images!

In [ ]:
# Computer Vision Introduction - How Computers See Images

print("=" * 60)
print("HOW COMPUTERS SEE IMAGES")
print("=" * 60)

# Load MNIST dataset (handwritten digits)
from sklearn.datasets import load_digits

digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"\nMNIST Digits dataset loaded:")
print(f"  Shape: {digits.images.shape}")
print(f"  Images: {len(digits.images)} samples of 8×8 pixels")
print(f"  Classes: {np.unique(y_digits).tolist()} (digits 0-9)")

# Visualize: Human view vs Computer view
fig, axes = plt.subplots(2, 5, figsize=(16, 7))

for i in range(5):
    idx = i * 100  # Sample every 100th image

    # Top row: Human view (image)
    axes[0, i].imshow(digits.images[idx], cmap="gray")
    axes[0, i].set_title(f'Human Sees:\nDigit "{y_digits[idx]}"', fontsize=12, fontweight="bold")
    axes[0, i].axis("off")

    # Bottom row: Computer view (numbers)
    im = axes[1, i].imshow(digits.images[idx], cmap="viridis", interpolation="none")
    axes[1, i].set_title(
        f"Computer Sees:\n64 numbers (8×8)", fontsize=12, fontweight="bold", color="red"
    )

    # Add pixel values as text
    for row in range(8):
        for col in range(8):
            value = int(digits.images[idx][row, col])
            color = "white" if value > 8 else "black"
            if row % 2 == 0 and col % 2 == 0:  # Show every other value to avoid clutter
                axes[1, i].text(
                    col, row, str(value), ha="center", va="center", fontsize=7, color=color
                )

    axes[1, i].set_xticks([])
    axes[1, i].set_yticks([])

plt.colorbar(im, ax=axes[1, :], label="Pixel Intensity (0-16)", shrink=0.8)
plt.suptitle("Human Perception vs Computer Representation", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

# RGB Color channels demonstration
print("\n" + "=" * 60)
print("RGB COLOR CHANNELS")
print("=" * 60)

# Create a simple colored image
img_rgb = np.zeros((100, 100, 3), dtype=np.uint8)
img_rgb[:, :33, 0] = 255  # Red channel (left third)
img_rgb[:, 33:66, 1] = 255  # Green channel (middle third)
img_rgb[:, 66:, 2] = 255  # Blue channel (right third)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original RGB image
axes[0].imshow(img_rgb)
axes[0].set_title("RGB Image\n(What we see)", fontsize=12, fontweight="bold")
axes[0].axis("off")

# Red channel
axes[1].imshow(img_rgb[:, :, 0], cmap="Reds")
axes[1].set_title("Red Channel\n(R values)", fontsize=12, fontweight="bold")
axes[1].axis("off")

# Green channel
axes[2].imshow(img_rgb[:, :, 1], cmap="Greens")
axes[2].set_title("Green Channel\n(G values)", fontsize=12, fontweight="bold")
axes[2].axis("off")

# Blue channel
axes[3].imshow(img_rgb[:, :, 2], cmap="Blues")
axes[3].set_title("Blue Channel\n(B values)", fontsize=12, fontweight="bold")
axes[3].axis("off")

plt.suptitle("RGB Image Decomposition: 3 Separate Matrices", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print("\nRGB Image shape: (height, width, 3)")
print(f"  Example: {img_rgb.shape}")
print(f"  Total values: {img_rgb.size} numbers")
print(f"  Memory: {img_rgb.nbytes} bytes")

# Visualize feature hierarchy
print("\n" + "=" * 60)
print("CNN FEATURE HIERARCHY (Conceptual)")
print("=" * 60)

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# Layer 1: Edges
edge_img = np.zeros((50, 50))
edge_img[20:30, :] = 1  # Horizontal edge
axes[0].imshow(edge_img, cmap="gray")
axes[0].set_title("Layer 1\nDetects EDGES\n(Low-level features)", fontsize=11, fontweight="bold")
axes[0].axis("off")

# Layer 2: Textures
texture = np.random.rand(50, 50)
axes[1].imshow(texture, cmap="gray")
axes[1].set_title("Layer 2\nDetects TEXTURES\n(Mid-level features)", fontsize=11, fontweight="bold")
axes[1].axis("off")

# Layer 3: Parts
axes[2].text(
    0.5,
    0.5,
    "Eyes\nWheels\nLeaves",
    ha="center",
    va="center",
    fontsize=14,
    bbox=dict(boxstyle="round", facecolor="lightblue"),
)
axes[2].set_xlim(0, 1)
axes[2].set_ylim(0, 1)
axes[2].set_title("Layer 3\nDetects PARTS\n(Object components)", fontsize=11, fontweight="bold")
axes[2].axis("off")

# Layer 4: Objects
axes[3].text(
    0.5,
    0.5,
    "Cat\nCar\nTree",
    ha="center",
    va="center",
    fontsize=16,
    fontweight="bold",
    bbox=dict(boxstyle="round", facecolor="lightgreen"),
)
axes[3].set_xlim(0, 1)
axes[3].set_ylim(0, 1)
axes[3].set_title("Layer 4\nDetects OBJECTS\n(High-level concepts)", fontsize=11, fontweight="bold")
axes[3].axis("off")

plt.suptitle(
    "How CNNs Build Understanding: Hierarchical Feature Learning", fontsize=14, fontweight="bold"
)
plt.tight_layout()
plt.show()

print("\n✓ Computer Vision fundamentals visualized!")
print("  • Images are matrices of numbers (pixels)")
print("  • RGB images have 3 channels (Red, Green, Blue)")
print("  • CNNs learn hierarchical features automatically")
print("  • Low-level (edges) → High-level (objects)")

## 2. Convolutional Neural Networks (CNNs)

**CNNs** are neural networks specifically designed for processing grid-like data (images).

### The Convolution Operation

**Convolution** slides a small filter (kernel) across the image to detect patterns.

**Analogy**: Like a flashlight scanning a dark room - you see one small area at a time.

### How Convolution Works

**Step-by-step:**
1. Take a **filter** (e.g., 3×3 matrix of weights)
2. **Slide** it across the image
3. At each position, compute **element-wise multiplication** and **sum**
4. Result: **Feature map** (activation map)

**Example**:
```
Image (5×5)        Filter (3×3)      Output (3×3)
[1 2 3 4 5]       [1 0 -1]
[0 1 2 3 4]   *   [1 0 -1]    =    Feature Map
[5 6 7 8 9]       [1 0 -1]
[2 3 4 5 6]
[1 2 3 4 5]
```

### Filters Detect Features

Different filters detect different patterns:

**Vertical Edge Detector:**
```
[1  0  -1]
[1  0  -1]
[1  0  -1]
```

**Horizontal Edge Detector:**
```
[ 1   1   1]
[ 0   0   0]
[-1  -1  -1]
```

**Blur Filter:**
```
[1/9  1/9  1/9]
[1/9  1/9  1/9]
[1/9  1/9  1/9]
```

### Key Parameters

**Filter Size (Kernel Size):**
- Common: 3×3, 5×5, 7×7
- Larger = more context, fewer parameters
- **Default: 3×3** (most common)

**Stride:**
- How many pixels to move filter each step
- Stride=1: Move 1 pixel (default)
- Stride=2: Move 2 pixels (downsample)

**Padding:**
- Add zeros around image border
- **Valid**: No padding (output smaller)
- **Same**: Pad so output = input size

### Output Size Formula

```
Output size = (Input size - Filter size + 2×Padding) / Stride + 1
```

**Example**:
- Input: 32×32, Filter: 5×5, Stride: 1, Padding: 0
- Output: (32 - 5 + 0) / 1 + 1 = 28×28

### Multiple Filters = Multiple Feature Maps

**One layer typically has many filters:**
- 32 filters → 32 feature maps
- 64 filters → 64 feature maps
- Each filter learns different patterns!

### Why CNNs Work

**1. Parameter Sharing:**
- Same filter used across entire image
- Detect "cat ear" anywhere in image
- Dramatically reduces parameters

**2. Local Connectivity:**
- Each neuron looks at small region
- Captures spatial relationships
- More efficient than fully connected

**3. Translation Invariance:**
- Detect patterns regardless of position
- Cat in corner or center → still detected

### CNN vs Fully Connected

**Fully Connected (28×28 image, 100 hidden):**
- Parameters: 28×28×100 = 78,400

**Conv Layer (32 filters of 3×3):**
- Parameters: 3×3×32 = 288 only!
- **272× fewer parameters!**

Let's visualize convolution in action!

In [ ]:
# Convolutional Neural Networks - Convolution in Action

print("=" * 60)
print("CONVOLUTION OPERATION DEMONSTRATION")
print("=" * 60)


# Manual convolution implementation
def convolve2d(image, kernel, stride=1, padding=0):
    """
    Perform 2D convolution on image with kernel.

    Parameters:
    - image: 2D numpy array
    - kernel: 2D numpy array (filter)
    - stride: step size for sliding window
    - padding: zeros to add around border

    Returns:
    - feature_map: convolution output
    """
    # Add padding
    if padding > 0:
        image = np.pad(image, padding, mode="constant", constant_values=0)

    # Get dimensions
    img_h, img_w = image.shape
    ker_h, ker_w = kernel.shape

    # Calculate output size
    out_h = (img_h - ker_h) // stride + 1
    out_w = (img_w - ker_w) // stride + 1

    # Initialize output
    feature_map = np.zeros((out_h, out_w))

    # Slide kernel across image
    for i in range(0, out_h):
        for j in range(0, out_w):
            # Extract region
            region = image[i * stride : i * stride + ker_h, j * stride : j * stride + ker_w]
            # Element-wise multiplication and sum
            feature_map[i, j] = np.sum(region * kernel)

    return feature_map


# Create sample image (simple pattern)
image = np.array(
    [
        [0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0],
    ],
    dtype=float,
)

# Define different filters
filters = {
    "Vertical Edge": np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]]),
    "Horizontal Edge": np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]]),
    "Blur": np.array([[1 / 9, 1 / 9, 1 / 9], [1 / 9, 1 / 9, 1 / 9], [1 / 9, 1 / 9, 1 / 9]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
}

print("\n1. DEMONSTRATING DIFFERENT FILTERS")
print("-" * 60)

# Apply filters and visualize
fig, axes = plt.subplots(2, 5, figsize=(18, 8))

# Original image
axes[0, 0].imshow(image, cmap="gray", vmin=-3, vmax=3)
axes[0, 0].set_title("Original Image\n(7×7)", fontsize=11, fontweight="bold")
axes[0, 0].axis("off")

axes[1, 0].text(
    0.5,
    0.5,
    "Input\nImage",
    ha="center",
    va="center",
    fontsize=14,
    fontweight="bold",
    bbox=dict(boxstyle="round", facecolor="lightblue"),
)
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_ylim(0, 1)
axes[1, 0].axis("off")

# Apply each filter
for idx, (name, kernel) in enumerate(filters.items(), 1):
    # Compute convolution
    feature_map = convolve2d(image, kernel, stride=1, padding=0)

    # Show filter
    axes[0, idx].imshow(kernel, cmap="RdBu", vmin=-1, vmax=1)
    axes[0, idx].set_title(f"{name} Filter\n(3×3)", fontsize=11, fontweight="bold")
    axes[0, idx].axis("off")

    # Add kernel values
    for i in range(3):
        for j in range(3):
            value = kernel[i, j]
            color = "white" if abs(value) > 0.5 else "black"
            axes[0, idx].text(
                j,
                i,
                f"{value:.1f}",
                ha="center",
                va="center",
                fontsize=9,
                color=color,
                fontweight="bold",
            )

    # Show feature map
    axes[1, idx].imshow(feature_map, cmap="RdBu", vmin=-3, vmax=3)
    axes[1, idx].set_title(
        f"Feature Map\n({feature_map.shape[0]}×{feature_map.shape[1]})",
        fontsize=11,
        fontweight="bold",
    )
    axes[1, idx].axis("off")

plt.suptitle(
    "Convolution: Different Filters Detect Different Features", fontsize=14, fontweight="bold"
)
plt.tight_layout()
plt.show()

print(f"\n✓ Applied 4 different filters to {image.shape} image")
print(f"  Filter size: 3×3")
print(f"  Output size: {feature_map.shape} (no padding, stride=1)")

# Demonstrate convolution step-by-step
print("\n2. STEP-BY-STEP CONVOLUTION PROCESS")
print("-" * 60)

# Use a digit image for realistic example
sample_digit = digits.images[0]  # First digit
vertical_filter = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Show original image
axes[0, 0].imshow(sample_digit, cmap="gray")
axes[0, 0].set_title("Original Digit\n(8×8 image)", fontsize=11, fontweight="bold")
axes[0, 0].axis("off")

# Show filter
axes[1, 0].imshow(vertical_filter, cmap="RdBu", vmin=-1, vmax=1)
axes[1, 0].set_title("Vertical Edge\nFilter (3×3)", fontsize=11, fontweight="bold")
for i in range(3):
    for j in range(3):
        axes[1, 0].text(
            j, i, vertical_filter[i, j], ha="center", va="center", fontsize=10, fontweight="bold"
        )
axes[1, 0].axis("off")

# Show 3 intermediate steps
positions = [(0, 0), (0, 3), (3, 3)]
for idx, (row, col) in enumerate(positions, 1):
    # Extract 3×3 region
    region = sample_digit[row : row + 3, col : col + 3]

    # Show region
    axes[0, idx].imshow(sample_digit, cmap="gray", alpha=0.3)
    # Highlight region with rectangle
    from matplotlib.patches import Rectangle

    rect = Rectangle((col - 0.5, row - 0.5), 3, 3, linewidth=3, edgecolor="red", facecolor="none")
    axes[0, idx].add_patch(rect)
    axes[0, idx].set_title(
        f"Step {idx}: Position ({row},{col})", fontsize=11, fontweight="bold", color="red"
    )
    axes[0, idx].axis("off")

    # Show computation
    result = np.sum(region * vertical_filter)
    axes[1, idx].imshow(region, cmap="gray")
    axes[1, idx].set_title(f"Region × Filter\n= {result:.1f}", fontsize=11, fontweight="bold")
    axes[1, idx].axis("off")

plt.suptitle("Convolution: Sliding Window Process", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Demonstrated sliding window at 3 positions")
print(f"  At each position: element-wise multiply + sum")

# Effect of stride and padding
print("\n3. EFFECT OF STRIDE AND PADDING")
print("-" * 60)

test_image = digits.images[5]  # Use a digit
edge_filter = filters["Vertical Edge"]

fig, axes = plt.subplots(2, 4, figsize=(18, 8))

# Original
axes[0, 0].imshow(test_image, cmap="gray")
axes[0, 0].set_title("Original\n8×8", fontsize=11, fontweight="bold")
axes[0, 0].axis("off")

# Configurations
configs = [("Stride=1, Pad=0", 1, 0), ("Stride=2, Pad=0", 2, 0), ("Stride=1, Pad=1", 1, 1)]

for idx, (title, stride, padding) in enumerate(configs, 1):
    # Compute convolution
    result = convolve2d(test_image, edge_filter, stride=stride, padding=padding)

    # Show result
    axes[0, idx].imshow(result, cmap="RdBu")
    axes[0, idx].set_title(
        f"{title}\nOutput: {result.shape[0]}×{result.shape[1]}", fontsize=11, fontweight="bold"
    )
    axes[0, idx].axis("off")

    # Show formula
    input_size = test_image.shape[0]
    filter_size = edge_filter.shape[0]
    output_size = (input_size - filter_size + 2 * padding) // stride + 1

    formula_text = f"Output = (I - F + 2P) / S + 1\n"
    formula_text += f"= ({input_size} - {filter_size} + 2×{padding}) / {stride} + 1\n"
    formula_text += f"= {output_size}×{output_size}"

    axes[1, idx].text(
        0.5,
        0.5,
        formula_text,
        ha="center",
        va="center",
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightyellow"),
        family="monospace",
    )
    axes[1, idx].set_xlim(0, 1)
    axes[1, idx].set_ylim(0, 1)
    axes[1, idx].set_title("Formula", fontsize=11, fontweight="bold")
    axes[1, idx].axis("off")

# Explanation
explanation = "Stride: Controls downsampling\n\n"
explanation += "Padding: Preserves border info\n\n"
explanation += "I=Input, F=Filter,\nP=Padding, S=Stride"
axes[1, 0].text(
    0.5,
    0.5,
    explanation,
    ha="center",
    va="center",
    fontsize=11,
    bbox=dict(boxstyle="round", facecolor="lightblue"),
)
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_ylim(0, 1)
axes[1, 0].axis("off")

plt.suptitle("Stride and Padding Control Output Dimensions", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Output size formula: (Input - Filter + 2×Padding) / Stride + 1")
print(f"  Stride=1, Pad=0: 8×8 → 6×6")
print(f"  Stride=2, Pad=0: 8×8 → 3×3 (downsampled)")
print(f"  Stride=1, Pad=1: 8×8 → 8×8 (same size)")

# Real-world example with MNIST
print("\n4. MULTIPLE FILTERS ON REAL DATA")
print("-" * 60)

# Load multiple digits
sample_indices = [0, 10, 20, 30]

fig, axes = plt.subplots(len(sample_indices), 5, figsize=(18, 14))

for row_idx, digit_idx in enumerate(sample_indices):
    digit_img = digits.images[digit_idx]
    digit_label = y_digits[digit_idx]

    # Original
    axes[row_idx, 0].imshow(digit_img, cmap="gray")
    axes[row_idx, 0].set_title(
        f"Digit {digit_label}" if row_idx == 0 else "", fontsize=11, fontweight="bold"
    )
    if row_idx == 0:
        axes[row_idx, 0].set_ylabel("Original", fontsize=11, fontweight="bold")
    axes[row_idx, 0].set_xticks([])
    axes[row_idx, 0].set_yticks([])

    # Apply 4 filters
    filter_names = ["Vertical Edge", "Horizontal Edge", "Blur", "Sharpen"]
    for col_idx, filter_name in enumerate(filter_names, 1):
        feature_map = convolve2d(digit_img, filters[filter_name], stride=1, padding=0)

        axes[row_idx, col_idx].imshow(feature_map, cmap="RdBu")
        if row_idx == 0:
            axes[row_idx, col_idx].set_title(filter_name, fontsize=11, fontweight="bold")
        axes[row_idx, col_idx].set_xticks([])
        axes[row_idx, col_idx].set_yticks([])

plt.suptitle(
    "Multiple Filters Extract Different Features from Same Image", fontsize=14, fontweight="bold"
)
plt.tight_layout()
plt.show()

print(f"\n✓ Applied 4 filters to 4 different digit images")
print(f"  Each filter detects different patterns")
print(f"  • Vertical Edge: Detects vertical boundaries")
print(f"  • Horizontal Edge: Detects horizontal boundaries")
print(f"  • Blur: Smooths the image")
print(f"  • Sharpen: Enhances edges")

# Parameter comparison
print("\n5. PARAMETER EFFICIENCY: CNN vs FULLY CONNECTED")
print("-" * 60)

# Calculate parameters
image_size = 28 * 28  # MNIST full size
hidden_units = 100

# Fully connected
fc_params = image_size * hidden_units
print(f"\nFully Connected Layer:")
print(f"  Input: {image_size} (28×28 image)")
print(f"  Hidden: {hidden_units} neurons")
print(f"  Parameters: {image_size} × {hidden_units} = {fc_params:,}")

# Convolutional
num_filters = 32
filter_size = 3
conv_params = num_filters * (filter_size * filter_size + 1)  # +1 for bias
print(f"\nConvolutional Layer:")
print(f"  Filters: {num_filters}")
print(f"  Filter size: {filter_size}×{filter_size}")
print(f"  Parameters: {num_filters} × ({filter_size}×{filter_size} + 1) = {conv_params}")

reduction = fc_params / conv_params
print(f"\n{'='*60}")
print(f"PARAMETER REDUCTION: {reduction:.0f}× fewer parameters!")
print(f"{'='*60}")

# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

categories = ["Fully Connected", "Convolutional"]
params = [fc_params, conv_params]
colors = ["#ff6b6b", "#4ecdc4"]

bars = ax.bar(categories, params, color=colors, edgecolor="black", linewidth=2)

# Add value labels
for bar, param in zip(bars, params):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{param:,}\nparameters",
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
    )

ax.set_ylabel("Number of Parameters", fontsize=13, fontweight="bold")
ax.set_title(
    f"Parameter Efficiency: CNN vs Fully Connected\n(CNN uses {reduction:.0f}× fewer parameters!)",
    fontsize=14,
    fontweight="bold",
)
ax.set_ylim(0, fc_params * 1.2)
ax.grid(axis="y", alpha=0.3, linestyle="--")

plt.tight_layout()
plt.show()

print("\n✓ Convolution operation demonstrated!")
print("  • Filters slide across image to detect patterns")
print("  • Different filters detect different features")
print("  • Stride and padding control output dimensions")
print("  • Massive parameter reduction vs fully connected layers")

## 3. Pooling and Padding

After convolution, we often apply **pooling** to reduce spatial dimensions and **padding** to control output size.

### What is Pooling?

**Pooling** (downsampling) reduces the size of feature maps while retaining important information.

**Purpose:**
1. Reduce computational cost (fewer parameters in next layer)
2. Make features more robust to small translations
3. Increase receptive field (neurons see larger regions)
4. Prevent overfitting

### Max Pooling

**Most common type**: Take the maximum value in each region.

**Example (2×2 Max Pooling):**
```
Input (4×4)          Output (2×2)
[1  3  2  4]         [3  4]
[5  6  1  2]    →    [8  7]
[7  8  3  1]
[4  2  7  5]
```

**How it works:**
- Slide a 2×2 window with stride=2
- At each position, take max value
- Output: 1/4 the size (16 → 4 pixels)

### Average Pooling

Take the average value instead of max.

**When to use:**
- Max Pooling: Most common, works well for CNNs
- Average Pooling: Sometimes used in later layers, preserves more info

### Common Pooling Configurations

**2×2 pooling, stride=2**: Most common (halves dimensions)
**3×3 pooling, stride=2**: More aggressive downsampling

### Pooling Effects

**Before pooling**: 224×224×64 feature maps
**After 2×2 pooling**: 112×112×64 feature maps
**Result**: 75% reduction in size! (50,176 → 12,544 pixels per channel)

### What is Padding?

**Padding** adds border pixels (usually zeros) around the image.

**Types:**

**Valid Padding** (no padding):
- Output smaller than input
- Border info lost

**Same Padding** (zero padding):
- Output same size as input (with stride=1)
- Preserves border information
- Most common in modern CNNs

**Example:**
```
Original (3×3)     Padded (5×5)
[1 2 3]           [0 0 0 0 0]
[4 5 6]     →     [0 1 2 3 0]
[7 8 9]           [0 4 5 6 0]
                  [0 7 8 9 0]
                  [0 0 0 0 0]
```

### Why Padding Matters

**Without padding:**
- Output shrinks with each layer
- Border pixels processed less
- Deep networks lose too much spatial info

**With padding:**
- Maintain spatial dimensions
- All pixels processed equally
- Build deeper networks

### Output Size with Padding & Pooling

**After Convolution:**
```
Output = (Input - Filter + 2×Padding) / Stride + 1
```

**After Pooling:**
```
Output = Input / Pool_size (typically)
```

**Example Pipeline:**
```
Input: 224×224
Conv (3×3, pad=1, stride=1): 224×224
Max Pool (2×2, stride=2): 112×112
Conv (3×3, pad=1, stride=1): 112×112
Max Pool (2×2, stride=2): 56×56
```

### Typical CNN Pattern

**CONV → ReLU → POOL → CONV → ReLU → POOL → FC**

Each CONV-POOL pair:
- CONV: Detects features
- ReLU: Non-linearity
- POOL: Reduces dimensions

### Pooling Advantages & Disadvantages

**Advantages:**
- ✓ Reduces computation
- ✓ Translation invariance
- ✓ Reduces overfitting
- ✓ Increases receptive field

**Disadvantages:**
- ✗ Loses spatial information
- ✗ Not learnable (fixed operation)
- ✗ May discard useful details

**Modern trend:** Some architectures avoid pooling, use strided convolutions instead!

Let's visualize pooling and padding operations!

In [ ]:
# Pooling and Padding - Demonstrations

print("=" * 60)
print("POOLING AND PADDING OPERATIONS")
print("=" * 60)


# Implement pooling operations
def max_pool2d(image, pool_size=2, stride=2):
    """Max pooling operation."""
    h, w = image.shape
    out_h = (h - pool_size) // stride + 1
    out_w = (w - pool_size) // stride + 1

    output = np.zeros((out_h, out_w))

    for i in range(out_h):
        for j in range(out_w):
            region = image[i * stride : i * stride + pool_size, j * stride : j * stride + pool_size]
            output[i, j] = np.max(region)

    return output


def avg_pool2d(image, pool_size=2, stride=2):
    """Average pooling operation."""
    h, w = image.shape
    out_h = (h - pool_size) // stride + 1
    out_w = (w - pool_size) // stride + 1

    output = np.zeros((out_h, out_w))

    for i in range(out_h):
        for j in range(out_w):
            region = image[i * stride : i * stride + pool_size, j * stride : j * stride + pool_size]
            output[i, j] = np.mean(region)

    return output


print("\n1. MAX POOLING vs AVERAGE POOLING")
print("-" * 60)

# Use a digit for demonstration
test_digit = digits.images[8]  # Pick an interesting digit

# Apply pooling
max_pooled = max_pool2d(test_digit, pool_size=2, stride=2)
avg_pooled = avg_pool2d(test_digit, pool_size=2, stride=2)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original
axes[0].imshow(test_digit, cmap="gray")
axes[0].set_title(
    f"Original Image\n{test_digit.shape[0]}×{test_digit.shape[1]} pixels",
    fontsize=12,
    fontweight="bold",
)
axes[0].axis("off")

# Max pooled
axes[1].imshow(max_pooled, cmap="gray")
axes[1].set_title(
    f"Max Pooling (2×2)\n{max_pooled.shape[0]}×{max_pooled.shape[1]} pixels\n(75% size reduction)",
    fontsize=12,
    fontweight="bold",
    color="red",
)
axes[1].axis("off")

# Average pooled
axes[2].imshow(avg_pooled, cmap="gray")
axes[2].set_title(
    f"Average Pooling (2×2)\n{avg_pooled.shape[0]}×{avg_pooled.shape[1]} pixels\n(75% size reduction)",
    fontsize=12,
    fontweight="bold",
    color="blue",
)
axes[2].axis("off")

plt.suptitle("Pooling: Downsampling Feature Maps", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Original: {test_digit.shape} → Pooled: {max_pooled.shape}")
print(f"  Size reduction: {(1 - max_pooled.size / test_digit.size) * 100:.1f}%")
print(f"  Max pooling preserves strong activations")
print(f"  Avg pooling smooths the feature map")

# Step-by-step max pooling
print("\n2. MAX POOLING STEP-BY-STEP")
print("-" * 60)

# Create a simple example for clarity
simple_image = np.array(
    [
        [1, 3, 2, 4, 1, 2],
        [5, 6, 1, 2, 3, 1],
        [7, 8, 3, 1, 2, 4],
        [4, 2, 7, 5, 1, 3],
        [3, 1, 4, 6, 2, 5],
        [2, 5, 1, 3, 7, 4],
    ],
    dtype=float,
)

fig, axes = plt.subplots(2, 4, figsize=(18, 9))

# Original image
axes[0, 0].imshow(simple_image, cmap="viridis", vmin=0, vmax=8)
axes[0, 0].set_title("Original\n6×6 Image", fontsize=11, fontweight="bold")
for i in range(6):
    for j in range(6):
        axes[0, 0].text(
            j,
            i,
            int(simple_image[i, j]),
            ha="center",
            va="center",
            color="white",
            fontsize=10,
            fontweight="bold",
        )
axes[0, 0].set_xticks([])
axes[0, 0].set_yticks([])

# Draw grid lines
for i in range(0, 7, 2):
    axes[0, 0].axhline(i - 0.5, color="red", linewidth=2)
    axes[0, 0].axvline(i - 0.5, color="red", linewidth=2)

# Show 3 pooling steps
positions = [(0, 0), (0, 2), (2, 2)]
for idx, (row, col) in enumerate(positions, 1):
    # Highlight region
    axes[0, idx].imshow(simple_image, cmap="viridis", alpha=0.3, vmin=0, vmax=8)

    # Draw all values
    for i in range(6):
        for j in range(6):
            axes[0, idx].text(
                j, i, int(simple_image[i, j]), ha="center", va="center", color="gray", fontsize=9
            )

    # Highlight current 2×2 region
    from matplotlib.patches import Rectangle

    rect = Rectangle((col - 0.5, row - 0.5), 2, 2, linewidth=4, edgecolor="red", facecolor="none")
    axes[0, idx].add_patch(rect)

    # Highlight values in region
    region = simple_image[row : row + 2, col : col + 2]
    for i in range(2):
        for j in range(2):
            axes[0, idx].text(
                col + j,
                row + i,
                int(region[i, j]),
                ha="center",
                va="center",
                color="red",
                fontsize=12,
                fontweight="bold",
            )

    max_val = np.max(region)
    axes[0, idx].set_title(
        f"Step {idx}\nRegion ({row}:{row+2}, {col}:{col+2})\nMax = {int(max_val)}",
        fontsize=11,
        fontweight="bold",
        color="red",
    )
    axes[0, idx].set_xticks([])
    axes[0, idx].set_yticks([])

# Final result
pooled_result = max_pool2d(simple_image, pool_size=2, stride=2)
axes[1, 0].imshow(pooled_result, cmap="viridis", vmin=0, vmax=8)
axes[1, 0].set_title("Pooled Output\n3×3 Result", fontsize=11, fontweight="bold")
for i in range(3):
    for j in range(3):
        axes[1, 0].text(
            j,
            i,
            int(pooled_result[i, j]),
            ha="center",
            va="center",
            color="white",
            fontsize=12,
            fontweight="bold",
        )
axes[1, 0].set_xticks([])
axes[1, 0].set_yticks([])

# Show the 3 extracted maxima
for idx in range(1, 4):
    row, col = positions[idx - 1]
    region = simple_image[row : row + 2, col : col + 2]
    max_val = np.max(region)

    axes[1, idx].text(
        0.5,
        0.5,
        f"Position ({row//2},{col//2})\n\n"
        f"2×2 Region:\n{region.astype(int).tolist()}\n\n"
        f"Max = {int(max_val)}",
        ha="center",
        va="center",
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightcoral"),
        family="monospace",
    )
    axes[1, idx].set_xlim(0, 1)
    axes[1, idx].set_ylim(0, 1)
    axes[1, idx].axis("off")

plt.suptitle("Max Pooling: Take Maximum from Each Region", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Max pooling with 2×2 filter, stride=2")
print(f"  Input: 6×6 → Output: 3×3")
print(f"  Each output pixel = max of 2×2 input region")

# Padding demonstration
print("\n3. PADDING DEMONSTRATION")
print("-" * 60)

# Use a small image for clarity
small_img = digits.images[0][:5, :5]  # 5×5 region

# Different padding amounts
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

padding_amounts = [0, 1, 2, 3]

for idx, pad in enumerate(padding_amounts):
    # Apply padding
    if pad > 0:
        padded = np.pad(small_img, pad, mode="constant", constant_values=0)
    else:
        padded = small_img.copy()

    # Show padded image
    axes[0, idx].imshow(padded, cmap="gray")
    axes[0, idx].set_title(
        f"Padding = {pad}\nSize: {padded.shape[0]}×{padded.shape[1]}",
        fontsize=11,
        fontweight="bold",
    )
    axes[0, idx].axis("off")

    # Highlight original region if padded
    if pad > 0:
        from matplotlib.patches import Rectangle

        rect = Rectangle(
            (pad - 0.5, pad - 0.5),
            small_img.shape[1],
            small_img.shape[0],
            linewidth=3,
            edgecolor="red",
            facecolor="none",
        )
        axes[0, idx].add_patch(rect)

    # Show formula
    original_size = small_img.shape[0]
    new_size = padded.shape[0]
    formula = f"Padded Size = Original + 2×Padding\n"
    formula += f"= {original_size} + 2×{pad}\n"
    formula += f"= {new_size}×{new_size}"

    axes[1, idx].text(
        0.5,
        0.5,
        formula,
        ha="center",
        va="center",
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightyellow"),
        family="monospace",
    )
    axes[1, idx].set_xlim(0, 1)
    axes[1, idx].set_ylim(0, 1)
    axes[1, idx].axis("off")

plt.suptitle("Padding: Add Zeros Around Border (shown in red box)", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Padding adds border pixels (usually zeros)")
print(f"  Padding=0: {small_img.shape} (no change)")
print(f"  Padding=1: {np.pad(small_img, 1, 'constant').shape}")
print(f"  Padding=2: {np.pad(small_img, 2, 'constant').shape}")

# Effect of padding on convolution output size
print("\n4. PADDING EFFECT ON OUTPUT SIZE")
print("-" * 60)

test_img = digits.images[3]
kernel_3x3 = filters["Vertical Edge"]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

padding_configs = [("Valid (Pad=0)", 0), ("Same (Pad=1)", 1), ("Full (Pad=2)", 2)]

for idx, (name, pad) in enumerate(padding_configs):
    # Apply convolution with padding
    conv_result = convolve2d(test_img, kernel_3x3, stride=1, padding=pad)

    # Show result
    axes[0, idx].imshow(conv_result, cmap="RdBu")

    size_change = "Same size!" if conv_result.shape == test_img.shape else "Size changed"
    axes[0, idx].set_title(
        f"{name}\nInput: {test_img.shape[0]}×{test_img.shape[1]} → Output: {conv_result.shape[0]}×{conv_result.shape[1]}\n{size_change}",
        fontsize=11,
        fontweight="bold",
    )
    axes[0, idx].axis("off")

    # Show explanation
    formula = f"Output Size Formula:\n"
    formula += f"(Input - Kernel + 2×Pad) / Stride + 1\n\n"
    formula += f"({test_img.shape[0]} - 3 + 2×{pad}) / 1 + 1\n"
    formula += f"= {conv_result.shape[0]}×{conv_result.shape[1]}\n\n"

    if pad == 0:
        desc = "No padding\n• Output shrinks\n• Border info lost"
    elif pad == 1:
        desc = "'Same' padding\n• Output = Input\n• Preserves dimensions"
    else:
        desc = "Extra padding\n• Output larger\n• Adds extra border"

    axes[1, idx].text(
        0.5,
        0.5,
        formula + desc,
        ha="center",
        va="center",
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightblue"),
        family="monospace",
    )
    axes[1, idx].set_xlim(0, 1)
    axes[1, idx].set_ylim(0, 1)
    axes[1, idx].axis("off")

plt.suptitle("Padding Controls Output Dimensions After Convolution", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Padding strategy affects network depth")
print(f"  Valid (pad=0): Output shrinks, limits network depth")
print(f"  Same (pad=1): Output = Input, enables deep networks")

# Full CNN pipeline visualization
print("\n5. COMPLETE CNN LAYER PIPELINE")
print("-" * 60)

# Start with original image
original = digits.images[5]

# Apply conv + pooling pipeline
conv_out = convolve2d(original, filters["Vertical Edge"], stride=1, padding=0)
relu_out = np.maximum(0, conv_out)  # ReLU activation
pool_out = max_pool2d(relu_out, pool_size=2, stride=2)

fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Step 1: Original
axes[0].imshow(original, cmap="gray")
axes[0].set_title(
    f"1. Input Image\n{original.shape[0]}×{original.shape[1]}", fontsize=11, fontweight="bold"
)
axes[0].axis("off")

# Arrow
axes[0].text(
    1.1,
    0.5,
    "→\nConv\n3×3",
    transform=axes[0].transAxes,
    fontsize=10,
    ha="center",
    va="center",
    bbox=dict(boxstyle="round", facecolor="lightgreen"),
)

# Step 2: After Convolution
axes[1].imshow(conv_out, cmap="RdBu")
axes[1].set_title(
    f"2. After Conv\n{conv_out.shape[0]}×{conv_out.shape[1]}", fontsize=11, fontweight="bold"
)
axes[1].axis("off")

axes[1].text(
    1.1,
    0.5,
    "→\nReLU",
    transform=axes[1].transAxes,
    fontsize=10,
    ha="center",
    va="center",
    bbox=dict(boxstyle="round", facecolor="lightyellow"),
)

# Step 3: After ReLU
axes[2].imshow(relu_out, cmap="RdBu")
axes[2].set_title(
    f"3. After ReLU\n{relu_out.shape[0]}×{relu_out.shape[1]}\n(negative → 0)",
    fontsize=11,
    fontweight="bold",
)
axes[2].axis("off")

axes[2].text(
    1.1,
    0.5,
    "→\nPool\n2×2",
    transform=axes[2].transAxes,
    fontsize=10,
    ha="center",
    va="center",
    bbox=dict(boxstyle="round", facecolor="lightcoral"),
)

# Step 4: After Pooling
axes[3].imshow(pool_out, cmap="RdBu")
axes[3].set_title(
    f"4. After MaxPool\n{pool_out.shape[0]}×{pool_out.shape[1]}\n(50% size)",
    fontsize=11,
    fontweight="bold",
)
axes[3].axis("off")

# Summary
axes[4].text(
    0.5,
    0.5,
    "Complete Layer:\n\n"
    "8×8 → Conv → 6×6\n"
    "↓ ReLU (no size change)\n"
    "6×6 → Pool → 3×3\n\n"
    f"Total reduction:\n"
    f"{original.size} → {pool_out.size} pixels\n"
    f"({(1-pool_out.size/original.size)*100:.0f}% smaller)",
    ha="center",
    va="center",
    fontsize=11,
    bbox=dict(boxstyle="round", facecolor="lightblue"),
    family="monospace",
)
axes[4].set_xlim(0, 1)
axes[4].set_ylim(0, 1)
axes[4].axis("off")

plt.suptitle("Typical CNN Layer: CONV → ReLU → POOL", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Complete CNN layer pipeline demonstrated!")
print(f"  1. Convolution: Detect features (8×8 → 6×6)")
print(f"  2. ReLU: Non-linearity (6×6 → 6×6)")
print(f"  3. MaxPool: Downsample (6×6 → 3×3)")
print(
    f"  Total: {original.size} → {pool_out.size} pixels ({(1-pool_out.size/original.size)*100:.0f}% reduction)"
)

# Compare different pooling sizes
print("\n6. EFFECT OF DIFFERENT POOL SIZES")
print("-" * 60)

test_image = digits.images[7]

fig, axes = plt.subplots(1, 4, figsize=(16, 5))

# Original
axes[0].imshow(test_image, cmap="gray")
axes[0].set_title(
    f"Original\n{test_image.shape[0]}×{test_image.shape[1]}", fontsize=11, fontweight="bold"
)
axes[0].axis("off")

# Different pool sizes
pool_configs = [(2, 2), (2, 4), (4, 4)]
for idx, (pool_size, stride) in enumerate(pool_configs, 1):
    pooled = max_pool2d(test_image, pool_size=pool_size, stride=stride)

    axes[idx].imshow(pooled, cmap="gray")
    reduction = (1 - pooled.size / test_image.size) * 100
    axes[idx].set_title(
        f"Pool {pool_size}×{pool_size}\n{pooled.shape[0]}×{pooled.shape[1]} ({reduction:.0f}% reduction)",
        fontsize=11,
        fontweight="bold",
    )
    axes[idx].axis("off")

plt.suptitle("Larger Pooling = More Aggressive Downsampling", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"\n✓ Pooling size controls downsampling rate")
print(f"  2×2 pool: Most common (4× reduction per layer)")
print(f"  4×4 pool: Aggressive (16× reduction per layer)")
print(f"  Trade-off: Computation vs information loss")

print("\n" + "=" * 60)
print("POOLING & PADDING SUMMARY")
print("=" * 60)
print("✓ Pooling reduces spatial dimensions (downsampling)")
print("  • Max pooling: Most common, preserves strong features")
print("  • Typically 2×2 with stride=2 (50% reduction)")
print("\n✓ Padding controls output size")
print("  • Valid (pad=0): Output shrinks")
print("  • Same (pad=1): Output = Input size")
print("  • Enables building deep networks")
print("\n✓ Typical pattern: CONV → ReLU → POOL")
print("=" * 60)

## 4. Building CNN Architecture

Now let's build a complete CNN from scratch and with Keras!

### Typical CNN Architecture

**Standard pattern:**
```
INPUT → [CONV → ReLU → POOL] × N → FC → FC → OUTPUT
```

**Example (LeNet-5 style for MNIST):**
```
Input (28×28×1)
    ↓
Conv1: 32 filters (3×3) → 28×28×32
ReLU → 28×28×32
MaxPool (2×2) → 14×14×32
    ↓
Conv2: 64 filters (3×3) → 14×14×64
ReLU → 14×14×64
MaxPool (2×2) → 7×7×64
    ↓
Flatten → 3136 neurons
Dense1 → 128 neurons
ReLU
Dropout (0.5)
    ↓
Dense2 (Output) → 10 neurons (softmax)
```

### CNN Layer Types

**1. Convolutional Layers (Feature Extraction)**
- Detect patterns (edges, textures, objects)
- Parameters: # filters, filter size, stride, padding
- Example: `Conv2D(32, (3,3), padding='same')`

**2. Activation Layers (Non-linearity)**
- ReLU most common: `f(x) = max(0, x)`
- Enables learning complex patterns
- No parameters (fixed function)

**3. Pooling Layers (Downsampling)**
- Reduce spatial dimensions
- MaxPooling most common
- Example: `MaxPooling2D((2,2))`

**4. Flatten Layer (Reshape)**
- Convert 2D feature maps → 1D vector
- Example: 7×7×64 → 3136 neurons
- Required before fully connected layers

**5. Dense (Fully Connected) Layers**
- Traditional neural network layers
- Final classification/regression
- Example: `Dense(10, activation='softmax')`

**6. Dropout (Regularization)**
- Randomly drop neurons during training
- Prevents overfitting
- Example: `Dropout(0.5)` drops 50%

### Famous CNN Architectures

| Architecture | Year | Depth | Key Innovation |
|--------------|------|-------|----------------|
| **LeNet-5** | 1998 | 5 | First successful CNN (digits) |
| **AlexNet** | 2012 | 8 | ReLU + GPU training (ImageNet winner) |
| **VGGNet** | 2014 | 16-19 | Deeper with 3×3 filters |
| **GoogLeNet** | 2014 | 22 | Inception modules (1×1 convs) |
| **ResNet** | 2015 | 50-152 | Skip connections (residual learning) |
| **MobileNet** | 2017 | - | Efficient for mobile devices |
| **EfficientNet** | 2019 | - | Optimized scaling |

### Calculating Parameters

**Convolutional Layer:**
```
Params = (filter_height × filter_width × input_channels + 1) × num_filters
```

**Example:**
- Input: 28×28×1
- Conv: 32 filters of 3×3
- Params = (3 × 3 × 1 + 1) × 32 = **320 parameters**

**Dense Layer:**
```
Params = (input_size + 1) × output_size
```

**Example:**
- Input: 3136 neurons
- Output: 128 neurons
- Params = (3136 + 1) × 128 = **401,536 parameters**

**Notice**: Most parameters are in Dense layers!

### Receptive Field

**Receptive field** = Region of input that affects one output neuron

**Grows with depth:**
- After Conv1 (3×3): 3×3 receptive field
- After Pool1 (2×2): 6×6 receptive field
- After Conv2 (3×3): 10×10 receptive field
- After Pool2 (2×2): 20×20 receptive field

**Deeper networks see more context!**

### Design Principles

**1. Start Small, Go Deep**
- More layers better than fewer wide layers
- 3×3 filters are sweet spot

**2. Double Filters, Halve Dimensions**
- Conv1: 32 filters, 28×28
- Pool1: 32 filters, 14×14
- Conv2: 64 filters, 14×14
- Pool2: 64 filters, 7×7

**3. Use Batch Normalization**
- Normalize activations between layers
- Faster training, better performance

**4. Add Dropout Before Dense Layers**
- Prevent overfitting
- Typically 0.25-0.5

**5. Data Augmentation**
- Rotate, flip, zoom images
- Dramatically improves generalization

### Common Mistakes

❌ **Too many parameters in Dense layers**
→ Add more CONV/POOL, reduce Dense size

❌ **Too deep too fast**
→ Gradually increase depth

❌ **No regularization**
→ Add Dropout, L2 regularization

❌ **Wrong output activation**
→ Binary: sigmoid, Multi-class: softmax

Let's build CNNs from scratch and with Keras!

In [ ]:
# Building CNN Architecture - Complete Implementation

print("=" * 60)
print("BUILDING CNN WITH KERAS/TENSORFLOW")
print("=" * 60)

# Import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, models
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.utils import to_categorical

    print(f"\n✓ TensorFlow {tf.__version__} loaded successfully!")
    has_tf = True
except ImportError:
    print("\n⚠ TensorFlow not available. Install with: pip install tensorflow")
    has_tf = False

if has_tf:
    # Load and prepare MNIST data
    print("\n" + "=" * 60)
    print("1. LOADING AND PREPARING MNIST DATA")
    print("=" * 60)

    # Load dataset
    (X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

    # Normalize pixel values to [0, 1]
    X_train_full = X_train_full.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0

    # Reshape for CNN (add channel dimension)
    X_train_full = X_train_full.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

    # Split into train and validation
    X_train = X_train_full[:50000]
    y_train = y_train_full[:50000]
    X_val = X_train_full[50000:]
    y_val = y_train_full[50000:]

    # Convert labels to one-hot encoding
    y_train_cat = to_categorical(y_train, 10)
    y_val_cat = to_categorical(y_val, 10)
    y_test_cat = to_categorical(y_test, 10)

    print(f"\n✓ Data loaded and preprocessed:")
    print(f"  Training: {X_train.shape}, Labels: {y_train_cat.shape}")
    print(f"  Validation: {X_val.shape}, Labels: {y_val_cat.shape}")
    print(f"  Test: {X_test.shape}, Labels: {y_test_cat.shape}")
    print(f"  Pixel range: [{X_train.min():.1f}, {X_train.max():.1f}]")
    print(f"  Classes: {np.unique(y_train).tolist()}")

    # Build CNN model
    print("\n" + "=" * 60)
    print("2. BUILDING CNN ARCHITECTURE")
    print("=" * 60)

    model = models.Sequential(
        [
            # First Conv Block
            layers.Conv2D(
                32, (3, 3), activation="relu", padding="same", input_shape=(28, 28, 1), name="conv1"
            ),
            layers.MaxPooling2D((2, 2), name="pool1"),
            # Second Conv Block
            layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="conv2"),
            layers.MaxPooling2D((2, 2), name="pool2"),
            # Flatten and Dense layers
            layers.Flatten(name="flatten"),
            layers.Dense(128, activation="relu", name="dense1"),
            layers.Dropout(0.5, name="dropout"),
            layers.Dense(10, activation="softmax", name="output"),
        ],
        name="MNIST_CNN",
    )

    # Display architecture
    print("\n✓ CNN Model Architecture:")
    print("-" * 60)
    model.summary()

    # Visualize architecture with text diagram
    print("\n" + "=" * 60)
    print("ARCHITECTURE VISUALIZATION")
    print("=" * 60)

    architecture_text = """
    Input Image (28×28×1)
           ↓
    ┌─────────────────────┐
    │ Conv2D (32 filters) │ → 28×28×32
    │ 3×3, padding='same' │
    │ ReLU activation     │
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ MaxPooling2D (2×2)  │ → 14×14×32
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ Conv2D (64 filters) │ → 14×14×64
    │ 3×3, padding='same' │
    │ ReLU activation     │
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ MaxPooling2D (2×2)  │ → 7×7×64
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ Flatten             │ → 3136
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ Dense (128)         │ → 128
    │ ReLU activation     │
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ Dropout (0.5)       │ → 128
    └─────────────────────┘
           ↓
    ┌─────────────────────┐
    │ Dense (10)          │ → 10
    │ Softmax activation  │
    └─────────────────────┘
           ↓
    Output Probabilities (10 classes)
    """
    print(architecture_text)

    # Count parameters by layer
    print("\n" + "=" * 60)
    print("PARAMETER BREAKDOWN")
    print("=" * 60)

    total_params = 0
    for layer in model.layers:
        params = layer.count_params()
        total_params += params
        print(f"{layer.name:12s}: {params:>10,} parameters")

    print("-" * 60)
    print(f"{'Total':12s}: {total_params:>10,} parameters")

    # Compile model
    print("\n" + "=" * 60)
    print("3. COMPILING MODEL")
    print("=" * 60)

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    print("\n✓ Model compiled with:")
    print("  Optimizer: Adam")
    print("  Loss: Categorical Crossentropy")
    print("  Metrics: Accuracy")

    # Train model
    print("\n" + "=" * 60)
    print("4. TRAINING MODEL")
    print("=" * 60)
    print("\nTraining CNN on MNIST (this may take 1-2 minutes)...")

    history = model.fit(
        X_train,
        y_train_cat,
        epochs=5,
        batch_size=128,
        validation_data=(X_val, y_val_cat),
        verbose=1,
    )

    # Evaluate on test set
    print("\n" + "=" * 60)
    print("5. EVALUATING MODEL")
    print("=" * 60)

    test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

    print(f"\n✓ Test Results:")
    print(f"  Loss: {test_loss:.4f}")
    print(f"  Accuracy: {test_accuracy*100:.2f}%")

    # Visualize training history
    print("\n" + "=" * 60)
    print("6. TRAINING HISTORY VISUALIZATION")
    print("=" * 60)

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    axes[0].plot(history.history["accuracy"], "b-", label="Training", linewidth=2)
    axes[0].plot(history.history["val_accuracy"], "r-", label="Validation", linewidth=2)
    axes[0].set_title("Model Accuracy", fontsize=13, fontweight="bold")
    axes[0].set_xlabel("Epoch", fontsize=11)
    axes[0].set_ylabel("Accuracy", fontsize=11)
    axes[0].legend(fontsize=10)
    axes[0].grid(alpha=0.3)

    # Plot loss
    axes[1].plot(history.history["loss"], "b-", label="Training", linewidth=2)
    axes[1].plot(history.history["val_loss"], "r-", label="Validation", linewidth=2)
    axes[1].set_title("Model Loss", fontsize=13, fontweight="bold")
    axes[1].set_xlabel("Epoch", fontsize=11)
    axes[1].set_ylabel("Loss", fontsize=11)
    axes[1].legend(fontsize=10)
    axes[1].grid(alpha=0.3)

    plt.suptitle("CNN Training Progress", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    # Make predictions
    print("\n" + "=" * 60)
    print("7. PREDICTION EXAMPLES")
    print("=" * 60)

    # Get predictions for first 12 test images
    predictions = model.predict(X_test[:12], verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = y_test[:12]

    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    axes = axes.ravel()

    for i in range(12):
        axes[i].imshow(X_test[i].reshape(28, 28), cmap="gray")

        pred_class = predicted_classes[i]
        true_class = true_classes[i]
        confidence = predictions[i][pred_class] * 100

        # Color code: green if correct, red if wrong
        color = "green" if pred_class == true_class else "red"

        axes[i].set_title(
            f"Pred: {pred_class}\nTrue: {true_class}\n({confidence:.1f}%)",
            fontsize=10,
            color=color,
            fontweight="bold",
        )
        axes[i].axis("off")

    plt.suptitle("CNN Predictions (Green=Correct, Red=Wrong)", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    correct_predictions = np.sum(predicted_classes == true_classes)
    print(f"\n✓ Predictions on 12 samples:")
    print(f"  Correct: {correct_predictions}/12 ({correct_predictions/12*100:.1f}%)")

    # Visualize learned filters
    print("\n" + "=" * 60)
    print("8. VISUALIZING LEARNED FILTERS")
    print("=" * 60)

    # Get first convolutional layer weights
    conv1_weights = model.layers[0].get_weights()[0]  # Shape: (3, 3, 1, 32)

    # Visualize first 16 filters
    fig, axes = plt.subplots(4, 8, figsize=(16, 8))

    for i in range(16):
        ax = axes[i // 8, i % 8]

        # Extract filter (3×3)
        filter_img = conv1_weights[:, :, 0, i]

        # Normalize for visualization
        filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min() + 1e-8)

        ax.imshow(filter_img, cmap="viridis")
        ax.set_title(f"Filter {i+1}", fontsize=9)
        ax.axis("off")

    # Hide unused subplots
    for i in range(16, 32):
        axes[i // 8, i % 8].axis("off")

    plt.suptitle("Learned Filters from First Conv Layer (3×3 each)", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    print(f"\n✓ Visualized 16 of 32 learned filters")
    print(f"  Each filter: 3×3 pixels")
    print(f"  These filters detect different edge patterns!")

    # Feature map visualization
    print("\n" + "=" * 60)
    print("9. FEATURE MAP VISUALIZATION")
    print("=" * 60)

    # Create model that outputs intermediate layers
    layer_outputs = [layer.output for layer in model.layers[:3]]  # conv1, pool1, conv2
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

    # Get activations for one test image
    test_img = X_test[0:1]  # First test image
    activations = activation_model.predict(test_img, verbose=0)

    # Visualize
    layer_names = ["Conv1 (32 filters)", "Pool1", "Conv2 (64 filters)"]

    for layer_idx, (activation, layer_name) in enumerate(zip(activations, layer_names)):
        num_features = min(8, activation.shape[-1])  # Show 8 feature maps

        fig, axes = plt.subplots(1, num_features + 1, figsize=(18, 3))

        # Show original image
        axes[0].imshow(test_img[0].reshape(28, 28), cmap="gray")
        axes[0].set_title("Input", fontsize=10, fontweight="bold")
        axes[0].axis("off")

        # Show feature maps
        for i in range(num_features):
            axes[i + 1].imshow(activation[0, :, :, i], cmap="viridis")
            axes[i + 1].set_title(f"Feature {i+1}", fontsize=10)
            axes[i + 1].axis("off")

        plt.suptitle(f"{layer_name} - Feature Maps", fontsize=12, fontweight="bold")
        plt.tight_layout()
        plt.show()

    print(f"\n✓ Feature maps visualized for 3 layers")
    print(f"  Different filters detect different patterns")
    print(f"  Early layers: edges, Late layers: complex patterns")

    print("\n" + "=" * 60)
    print("CNN BUILDING SUMMARY")
    print("=" * 60)
    print(f"✓ Built CNN with {total_params:,} parameters")
    print(f"✓ Trained for 5 epochs")
    print(f"✓ Achieved {test_accuracy*100:.2f}% test accuracy")
    print(f"✓ Architecture: Conv→Pool→Conv→Pool→Dense→Output")
    print("=" * 60)

else:
    print("\nSkipping CNN building (TensorFlow not available)")
    print("Install TensorFlow to run this section:")
    print("  pip install tensorflow")

## 5. Transfer Learning with Pre-trained Models

**Transfer Learning**: Use models pre-trained on large datasets for your own tasks!

### What is Transfer Learning?

Instead of training from scratch, start with a model already trained on millions of images.

**Analogy**: Like hiring an experienced artist instead of teaching someone from scratch.

### Why Transfer Learning?

**Advantages:**
1. **Less data needed**: Works with small datasets (100s instead of millions)
2. **Faster training**: Hours instead of days/weeks
3. **Better performance**: Pre-trained features are powerful
4. **Lower cost**: No need for expensive GPUs for weeks

**When to use:**
- ✅ Small dataset (< 10,000 images)
- ✅ Limited computational resources
- ✅ Similar to ImageNet classes (objects, animals, scenes)
- ✅ Need quick prototype

### Popular Pre-trained Models

| Model | Year | Size | ImageNet Accuracy | Parameters |
|-------|------|------|-------------------|------------|
| **VGG16** | 2014 | 528 MB | 71.3% | 138M |
| **ResNet50** | 2015 | 98 MB | 74.9% | 25.6M |
| **InceptionV3** | 2015 | 92 MB | 77.9% | 23.8M |
| **MobileNetV2** | 2018 | 14 MB | 71.3% | 3.5M (mobile!) |
| **EfficientNetB0** | 2019 | 29 MB | 77.1% | 5.3M |

**ImageNet**: 1.2M images, 1000 classes (dogs, cats, cars, etc.)

### Two Transfer Learning Strategies

**1. Feature Extraction**
- Freeze pre-trained layers (don't update weights)
- Add new classifier on top
- Only train new layers

```python
base_model.trainable = False  # Freeze all layers
```

**Use when:**
- Very small dataset (< 1000 images)
- Very similar to ImageNet

**2. Fine-Tuning**
- Unfreeze some top layers
- Train entire model with small learning rate
- Allows adaptation to your specific data

```python
base_model.trainable = True  # Unfreeze
# Freeze early layers, train later layers
```

**Use when:**
- Medium dataset (1000-10,000 images)
- Somewhat different from ImageNet

### Transfer Learning Workflow

**Step 1**: Load pre-trained model (without top classification layer)
```python
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,  # Remove classification layer
    weights='imagenet',  # Use ImageNet weights
    input_shape=(224, 224, 3)
)
```

**Step 2**: Freeze base model
```python
base_model.trainable = False
```

**Step 3**: Add custom classifier
```python
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
```

**Step 4**: Train
```python
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(X_train, y_train, epochs=10)
```

**Step 5** (Optional): Fine-tune
```python
base_model.trainable = True
# Freeze first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False
    
# Recompile with small learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy')
model.fit(X_train, y_train, epochs=10)
```

### Data Preprocessing for Pre-trained Models

**Important**: Different models trained with different preprocessing!

```python
# VGG16, ResNet
from tensorflow.keras.applications.resnet50 import preprocess_input

# MobileNet
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Always use model's specific preprocessing!
X_processed = preprocess_input(X)
```

### Input Size Requirements

**Most models expect specific sizes:**
- VGG16, ResNet: 224×224
- InceptionV3: 299×299
- EfficientNet: varies (224-600)

**Resize images:**
```python
tf.keras.layers.Resizing(224, 224)
```

### Example Performance Comparison

| Approach | Training Time | Accuracy (1000 images) |
|----------|---------------|------------------------|
| From Scratch | 2 hours | 65% |
| Transfer Learning (frozen) | 10 minutes | 85% |
| Transfer Learning (fine-tuned) | 30 minutes | 92% |

**Transfer learning is 12× faster and 27% more accurate!**

### Best Practices

**1. Start with feature extraction**
- Train only classifier first
- Verify it works before fine-tuning

**2. Use small learning rate for fine-tuning**
- 10-100× smaller than normal
- Prevents destroying pre-trained weights

**3. Unfreeze gradually**
- Don't unfreeze all layers at once
- Start with last few layers

**4. Monitor validation accuracy**
- Stop if overfitting
- Use early stopping

**5. Data augmentation**
- Even with transfer learning!
- Helps prevent overfitting

Let's implement transfer learning with MobileNetV2!

In [ ]:
# Transfer Learning - Implementation with MobileNetV2

print("=" * 60)
print("TRANSFER LEARNING DEMONSTRATION")
print("=" * 60)

if has_tf:
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    from tensorflow.keras.layers import GlobalAveragePooling2D
    from tensorflow.keras.optimizers import Adam
    import time

    print("\n" + "=" * 60)
    print("1. PREPARING DATA FOR TRANSFER LEARNING")
    print("=" * 60)

    # Note: MobileNetV2 expects 3-channel RGB images (224×224×3)
    # We'll convert MNIST grayscale to RGB by repeating channels

    # Take a subset for faster demonstration
    X_train_subset = X_train[:5000]
    y_train_subset = y_train_cat[:5000]
    X_val_subset = X_val[:1000]
    y_val_subset = y_val_cat[:1000]

    # Resize MNIST from 28×28 to 32×32 (MobileNet minimum)
    # Convert grayscale to RGB (repeat channel 3 times)
    def prepare_for_transfer_learning(X):
        # Resize to 32×32
        X_resized = tf.image.resize(X, [32, 32])
        # Convert to RGB (repeat grayscale channel)
        X_rgb = tf.image.grayscale_to_rgb(X_resized)
        return X_rgb.numpy()

    print("\nPreparing images for MobileNetV2...")
    X_train_rgb = prepare_for_transfer_learning(X_train_subset)
    X_val_rgb = prepare_for_transfer_learning(X_val_subset)

    # Apply MobileNetV2 preprocessing
    X_train_preprocessed = preprocess_input(X_train_rgb.copy())
    X_val_preprocessed = preprocess_input(X_val_rgb.copy())

    print(f"\n✓ Data prepared:")
    print(f"  Training shape: {X_train_preprocessed.shape}")
    print(f"  Validation shape: {X_val_preprocessed.shape}")
    print(f"  Converted: 28×28×1 → 32×32×3 (RGB)")

    # Build transfer learning model
    print("\n" + "=" * 60)
    print("2. BUILDING TRANSFER LEARNING MODEL")
    print("=" * 60)

    # Load pre-trained MobileNetV2
    print("\nLoading MobileNetV2 with ImageNet weights...")
    base_model = MobileNetV2(
        include_top=False,  # Exclude classification layer
        weights="imagenet",  # Use ImageNet weights
        input_shape=(32, 32, 3),
    )

    # Freeze base model
    base_model.trainable = False

    print(f"\n✓ MobileNetV2 loaded:")
    print(f"  Trained on: ImageNet (1.2M images, 1000 classes)")
    print(f"  Parameters: {base_model.count_params():,}")
    print(f"  Frozen: Yes (we'll only train classifier)")

    # Build full model
    transfer_model = models.Sequential(
        [
            base_model,
            GlobalAveragePooling2D(name="global_pool"),
            layers.Dense(128, activation="relu", name="dense1"),
            layers.Dropout(0.5, name="dropout"),
            layers.Dense(10, activation="softmax", name="output"),
        ],
        name="Transfer_Learning_Model",
    )

    print("\n✓ Complete model architecture:")
    print("-" * 60)
    transfer_model.summary()

    # Count trainable vs non-trainable
    total_params = transfer_model.count_params()
    trainable_params = sum([tf.size(w).numpy() for w in transfer_model.trainable_weights])
    non_trainable_params = total_params - trainable_params

    print("\n" + "=" * 60)
    print("PARAMETER BREAKDOWN")
    print("=" * 60)
    print(f"Total parameters:        {total_params:>12,}")
    print(
        f"Trainable parameters:    {trainable_params:>12,}  ({trainable_params/total_params*100:.1f}%)"
    )
    print(
        f"Non-trainable parameters:{non_trainable_params:>12,}  ({non_trainable_params/total_params*100:.1f}%)"
    )
    print("\n✓ Only training {:.1f}% of parameters!".format(trainable_params / total_params * 100))

    # Compile model
    print("\n" + "=" * 60)
    print("3. COMPILING TRANSFER LEARNING MODEL")
    print("=" * 60)

    transfer_model.compile(
        optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"]
    )

    print("\n✓ Model compiled with:")
    print("  Optimizer: Adam (lr=0.001)")
    print("  Loss: Categorical Crossentropy")

    # Train model
    print("\n" + "=" * 60)
    print("4. TRAINING (FEATURE EXTRACTION)")
    print("=" * 60)
    print("\nTraining only the classifier (base model frozen)...")

    start_time = time.time()
    history_transfer = transfer_model.fit(
        X_train_preprocessed,
        y_train_subset,
        epochs=5,
        batch_size=64,
        validation_data=(X_val_preprocessed, y_val_subset),
        verbose=1,
    )
    training_time = time.time() - start_time

    print(f"\n✓ Training completed in {training_time:.1f} seconds")

    # Evaluate
    val_loss, val_acc = transfer_model.evaluate(X_val_preprocessed, y_val_subset, verbose=0)
    print(f"  Validation Accuracy: {val_acc*100:.2f}%")

    # Compare with custom CNN
    print("\n" + "=" * 60)
    print("5. PERFORMANCE COMPARISON")
    print("=" * 60)

    print("\n📊 Transfer Learning vs Custom CNN:\n")
    print(f"{'Metric':<25} {'Custom CNN':>15} {'Transfer Learning':>20}")
    print("-" * 62)

    # Training time comparison (approximate)
    custom_time = 120  # Approximate from earlier
    print(f"{'Training Time':<25} {custom_time:>12.0f}s {training_time:>18.1f}s")

    # Accuracy comparison
    custom_acc = test_accuracy * 100  # From earlier CNN
    transfer_acc = val_acc * 100
    print(f"{'Accuracy':<25} {custom_acc:>12.1f}% {transfer_acc:>17.1f}%")

    # Parameters comparison
    custom_params = model.count_params()
    transfer_trainable = trainable_params
    print(f"{'Trainable Params':<25} {custom_params:>12,} {transfer_trainable:>18,}")

    print("\n" + "=" * 60)
    speedup = custom_time / training_time
    print(f"⚡ Transfer learning is {speedup:.1f}× faster!")
    print("=" * 60)

    # Visualize training history
    print("\n" + "=" * 60)
    print("6. TRAINING HISTORY")
    print("=" * 60)

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Accuracy
    axes[0].plot(history_transfer.history["accuracy"], "b-", label="Training", linewidth=2)
    axes[0].plot(history_transfer.history["val_accuracy"], "r-", label="Validation", linewidth=2)
    axes[0].set_title("Transfer Learning: Accuracy", fontsize=13, fontweight="bold")
    axes[0].set_xlabel("Epoch", fontsize=11)
    axes[0].set_ylabel("Accuracy", fontsize=11)
    axes[0].legend(fontsize=10)
    axes[0].grid(alpha=0.3)

    # Loss
    axes[1].plot(history_transfer.history["loss"], "b-", label="Training", linewidth=2)
    axes[1].plot(history_transfer.history["val_loss"], "r-", label="Validation", linewidth=2)
    axes[1].set_title("Transfer Learning: Loss", fontsize=13, fontweight="bold")
    axes[1].set_xlabel("Epoch", fontsize=11)
    axes[1].set_ylabel("Loss", fontsize=11)
    axes[1].legend(fontsize=10)
    axes[1].grid(alpha=0.3)

    plt.suptitle("Transfer Learning Training Progress", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    # Make predictions
    print("\n" + "=" * 60)
    print("7. PREDICTIONS WITH TRANSFER LEARNING")
    print("=" * 60)

    # Prepare test images
    X_test_rgb = prepare_for_transfer_learning(X_test[:12])
    X_test_preprocessed = preprocess_input(X_test_rgb.copy())

    predictions_transfer = transfer_model.predict(X_test_preprocessed, verbose=0)
    predicted_classes_transfer = np.argmax(predictions_transfer, axis=1)
    true_classes_transfer = y_test[:12]

    fig, axes = plt.subplots(2, 6, figsize=(18, 6))
    axes = axes.ravel()

    for i in range(12):
        axes[i].imshow(X_test[i].reshape(28, 28), cmap="gray")

        pred_class = predicted_classes_transfer[i]
        true_class = true_classes_transfer[i]
        confidence = predictions_transfer[i][pred_class] * 100

        color = "green" if pred_class == true_class else "red"

        axes[i].set_title(
            f"Pred: {pred_class}\nTrue: {true_class}\n({confidence:.1f}%)",
            fontsize=10,
            color=color,
            fontweight="bold",
        )
        axes[i].axis("off")

    plt.suptitle(
        "Transfer Learning Predictions (Green=Correct, Red=Wrong)", fontsize=14, fontweight="bold"
    )
    plt.tight_layout()
    plt.show()

    correct = np.sum(predicted_classes_transfer == true_classes_transfer)
    print(f"\n✓ Accuracy on 12 samples: {correct}/12 ({correct/12*100:.1f}%)")

    # Fine-tuning demonstration
    print("\n" + "=" * 60)
    print("8. FINE-TUNING (OPTIONAL ADVANCED STEP)")
    print("=" * 60)

    print("\nFine-tuning: Unfreeze last few layers and retrain with small LR")
    print("\nArchitecture of base model:")
    print(f"  Total layers: {len(base_model.layers)}")
    print(f"  Currently all frozen: {not base_model.trainable}")

    # Unfreeze last 20 layers
    base_model.trainable = True
    fine_tune_at = len(base_model.layers) - 20

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Count trainable params after unfreezing
    trainable_after = sum([tf.size(w).numpy() for w in transfer_model.trainable_weights])

    print(f"\n✓ Fine-tuning setup:")
    print(f"  Unfrozen layers: Last {len(base_model.layers) - fine_tune_at} layers")
    print(f"  Frozen layers: First {fine_tune_at} layers")
    print(f"  Trainable parameters: {trainable_after:,} (was {trainable_params:,})")

    # Recompile with smaller learning rate
    transfer_model.compile(
        optimizer=Adam(learning_rate=0.0001),  # 10× smaller LR
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    print("\n✓ Recompiled with smaller learning rate:")
    print("  LR: 0.0001 (10× smaller to preserve pre-trained weights)")

    print("\n💡 Fine-tuning tips:")
    print("  • Use much smaller learning rate (0.0001 vs 0.001)")
    print("  • Unfreeze gradually (last layers first)")
    print("  • Monitor validation loss carefully")
    print("  • Stop if overfitting occurs")

    # Visualize architecture
    print("\n" + "=" * 60)
    print("9. TRANSFER LEARNING ARCHITECTURE")
    print("=" * 60)

    fig, ax = plt.subplots(figsize=(12, 8))

    architecture_viz = """
    ┌─────────────────────────────────────┐
    │         INPUT IMAGE (32×32×3)        │
    └─────────────────────────────────────┘
                     ↓
    ┌─────────────────────────────────────┐
    │      MobileNetV2 Base Model         │
    │     (Pre-trained on ImageNet)       │
    │                                      │
    │   ┌───────────────────────────┐     │
    │   │ Convolutional Blocks      │     │
    │   │ (155 layers)              │     │
    │   │                           │     │
    │   │ Learned features:         │     │
    │   │  • Edges & textures       │     │
    │   │  • Patterns & shapes      │     │
    │   │  • Object parts           │     │
    │   └───────────────────────────┘     │
    │                                      │
    │   🔒 FROZEN (weights not updated)   │
    │   2.2M parameters                    │
    └─────────────────────────────────────┘
                     ↓
    ┌─────────────────────────────────────┐
    │     Global Average Pooling 2D       │
    │          (Reduce spatial dims)      │
    └─────────────────────────────────────┘
                     ↓
    ┌─────────────────────────────────────┐
    │          Dense (128, ReLU)          │
    │      🔓 TRAINABLE                   │
    └─────────────────────────────────────┘
                     ↓
    ┌─────────────────────────────────────┐
    │          Dropout (0.5)              │
    └─────────────────────────────────────┘
                     ↓
    ┌─────────────────────────────────────┐
    │      Dense (10, Softmax)            │
    │      🔓 TRAINABLE                   │
    │      (Our custom classifier)        │
    └─────────────────────────────────────┘
                     ↓
           OUTPUT (10 classes)
    
    Key:
    🔒 = Frozen (use pre-trained weights)
    🔓 = Trainable (learn from our data)
    """

    ax.text(
        0.5,
        0.5,
        architecture_viz,
        ha="center",
        va="center",
        fontsize=10,
        family="monospace",
        bbox=dict(boxstyle="round", facecolor="lightblue", alpha=0.8),
    )
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")
    ax.set_title("Transfer Learning Architecture", fontsize=14, fontweight="bold")

    plt.tight_layout()
    plt.show()

    print("\n" + "=" * 60)
    print("TRANSFER LEARNING SUMMARY")
    print("=" * 60)
    print(f"✓ Used pre-trained MobileNetV2 from ImageNet")
    print(f"✓ Froze base model ({non_trainable_params:,} parameters)")
    print(f"✓ Trained only classifier ({trainable_params:,} parameters)")
    print(f"✓ Achieved {val_acc*100:.2f}% accuracy in {training_time:.1f}s")
    print(f"✓ {speedup:.1f}× faster than training from scratch!")
    print("\n💡 Transfer learning ideal for:")
    print("  • Small datasets (< 10K images)")
    print("  • Limited GPU resources")
    print("  • Quick prototypes")
    print("  • Similar to ImageNet classes")
    print("=" * 60)

else:
    print("\nSkipping transfer learning (TensorFlow not available)")
    print("Install TensorFlow to run this section")

## 6. Image Classification Project

Put everything together in a complete image classification pipeline!

### Project Overview

**Goal**: Build an end-to-end image classifier

**Steps:**
1. Load and explore data
2. Preprocess images
3. Build/train model
4. Evaluate performance
5. Make predictions
6. Analyze errors

### Real-World Classification Tasks

**Medical Imaging:**
- Detect tumors in X-rays/MRIs
- Classify skin lesions (melanoma detection)
- Diagnose diabetic retinopathy from eye scans

**Agriculture:**
- Identify plant diseases from leaf images
- Count crops from drone images
- Detect pests

**Manufacturing:**
- Quality control (detect defects)
- Sort products by type
- Safety monitoring

**Security:**
- Face recognition
- License plate reading
- Surveillance analysis

### Key Components of Classification Pipeline

**1. Data Loading & Exploration**
```python
# Load data
train_images, train_labels = load_dataset()

# Explore
print(f"Samples: {len(train_images)}")
print(f"Image shape: {train_images[0].shape}")
print(f"Classes: {np.unique(train_labels)}")
```

**2. Data Preprocessing**
```python
# Normalize pixels to [0, 1]
X = X.astype('float32') / 255.0

# Reshape for CNN if needed
X = X.reshape(-1, height, width, channels)

# One-hot encode labels
y = to_categorical(y, num_classes)
```

**3. Data Splitting**
```python
# Split into train/validation/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)
```

**4. Model Building**
```python
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
```

**5. Training**
```python
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val)
)
```

**6. Evaluation**
```python
# Test set evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
print(classification_report(y_true, y_pred_classes))
```

### Performance Metrics

**Accuracy**: Overall correct predictions
```
Accuracy = (TP + TN) / Total
```

**Precision**: Of positive predictions, how many correct?
```
Precision = TP / (TP + FP)
```

**Recall**: Of actual positives, how many found?
```
Recall = TP / (TP + FN)
```

**F1-Score**: Harmonic mean of precision and recall
```
F1 = 2 × (Precision × Recall) / (Precision + Recall)
```

### Common Issues & Solutions

**Problem: Overfitting**
- Validation accuracy lower than training
- Solution: Add Dropout, L2 regularization, data augmentation

**Problem: Underfitting**
- Both train and validation accuracy low
- Solution: Increase model capacity, train longer

**Problem: Class Imbalance**
- Some classes have many more samples
- Solution: Class weights, oversampling, focal loss

**Problem: Low Accuracy**
- Model not learning
- Solution: Check data quality, try transfer learning, tune hyperparameters

Let's build a complete classification project!

In [ ]:
# Image Classification Project - Complete Pipeline

print("=" * 60)
print("COMPLETE IMAGE CLASSIFICATION PROJECT")
print("=" * 60)

if has_tf:
    from sklearn.metrics import confusion_matrix, classification_report

    # Use our already-trained model from Section 4
    print("\n✓ Using CNN model trained earlier")
    print(f"  Model: {model.name}")
    print(f"  Test Accuracy: {test_accuracy*100:.2f}%")

    # Make predictions on test set
    print("\n" + "=" * 60)
    print("1. COMPREHENSIVE EVALUATION")
    print("=" * 60)

    # Get predictions
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_true_classes = y_test

    # Classification report
    print("\nClassification Report:")
    print("-" * 60)
    print(
        classification_report(
            y_true_classes, y_pred_classes, target_names=[str(i) for i in range(10)]
        )
    )

    # Confusion matrix
    print("\n" + "=" * 60)
    print("2. CONFUSION MATRIX")
    print("=" * 60)

    cm = confusion_matrix(y_true_classes, y_pred_classes)

    fig, ax = plt.subplots(figsize=(12, 10))

    im = ax.imshow(cm, cmap="Blues")

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("Count", fontsize=12, fontweight="bold")

    # Set ticks
    ax.set_xticks(np.arange(10))
    ax.set_yticks(np.arange(10))
    ax.set_xticklabels(range(10), fontsize=11)
    ax.set_yticklabels(range(10), fontsize=11)

    # Add labels
    ax.set_xlabel("Predicted Label", fontsize=13, fontweight="bold")
    ax.set_ylabel("True Label", fontsize=13, fontweight="bold")
    ax.set_title("Confusion Matrix: MNIST Digit Classification", fontsize=14, fontweight="bold")

    # Add text annotations
    for i in range(10):
        for j in range(10):
            text_color = "white" if cm[i, j] > cm.max() / 2 else "black"
            ax.text(
                j,
                i,
                cm[i, j],
                ha="center",
                va="center",
                color=text_color,
                fontsize=11,
                fontweight="bold",
            )

    plt.tight_layout()
    plt.show()

    # Per-class accuracy
    print("\n✓ Confusion Matrix Analysis:")
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    for digit in range(10):
        print(
            f"  Digit {digit}: {class_accuracies[digit]*100:.2f}% accuracy ({cm[digit, digit]}/{cm[digit].sum()} correct)"
        )

    # Error analysis
    print("\n" + "=" * 60)
    print("3. ERROR ANALYSIS")
    print("=" * 60)

    # Find misclassified examples
    misclassified_idx = np.where(y_pred_classes != y_true_classes)[0]

    print(
        f"\n✓ Total errors: {len(misclassified_idx)}/{len(y_test)} ({len(misclassified_idx)/len(y_test)*100:.2f}%)"
    )

    # Show worst predictions (lowest confidence on wrong predictions)
    misclassified_probs = y_pred_probs[misclassified_idx]
    misclassified_confidences = np.max(misclassified_probs, axis=1)

    # Sort by confidence (most confident errors first)
    sorted_idx = np.argsort(-misclassified_confidences)[:12]
    worst_errors = misclassified_idx[sorted_idx]

    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.ravel()

    for idx, error_idx in enumerate(worst_errors):
        axes[idx].imshow(X_test[error_idx].reshape(28, 28), cmap="gray")

        true_label = y_true_classes[error_idx]
        pred_label = y_pred_classes[error_idx]
        confidence = y_pred_probs[error_idx][pred_label] * 100

        axes[idx].set_title(
            f"True: {true_label} | Pred: {pred_label}\nConfidence: {confidence:.1f}%",
            fontsize=11,
            fontweight="bold",
            color="red",
        )
        axes[idx].axis("off")

    plt.suptitle("Most Confident Misclassifications", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    print("\n✓ Showing 12 most confident misclassifications")
    print("  These are errors where model was very confident but wrong!")

    # Per-class error analysis
    print("\n" + "=" * 60)
    print("4. MOST CONFUSED DIGIT PAIRS")
    print("=" * 60)

    # Find most common confusions (off-diagonal elements)
    confusion_pairs = []
    for i in range(10):
        for j in range(10):
            if i != j and cm[i, j] > 0:
                confusion_pairs.append((i, j, cm[i, j]))

    # Sort by count
    confusion_pairs.sort(key=lambda x: x[2], reverse=True)

    print("\nTop 10 most common misclassifications:\n")
    print(f"{'True':<6} {'Predicted':<12} {'Count':<8} {'% of true class'}")
    print("-" * 50)

    for true_label, pred_label, count in confusion_pairs[:10]:
        pct = count / cm[true_label].sum() * 100
        print(f"{true_label:<6} {pred_label:<12} {count:<8} {pct:>6.2f}%")

    # Prediction confidence distribution
    print("\n" + "=" * 60)
    print("5. PREDICTION CONFIDENCE ANALYSIS")
    print("=" * 60)

    # Get max probabilities for all predictions
    all_confidences = np.max(y_pred_probs, axis=1)

    # Separate correct vs incorrect
    correct_mask = y_pred_classes == y_true_classes
    correct_confidences = all_confidences[correct_mask]
    incorrect_confidences = all_confidences[~correct_mask]

    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Histogram of confidences
    axes[0].hist(
        correct_confidences, bins=50, alpha=0.7, label="Correct", color="green", edgecolor="black"
    )
    axes[0].hist(
        incorrect_confidences, bins=50, alpha=0.7, label="Incorrect", color="red", edgecolor="black"
    )
    axes[0].set_xlabel("Confidence", fontsize=12, fontweight="bold")
    axes[0].set_ylabel("Count", fontsize=12, fontweight="bold")
    axes[0].set_title("Prediction Confidence Distribution", fontsize=13, fontweight="bold")
    axes[0].legend(fontsize=11)
    axes[0].grid(alpha=0.3)

    # Box plot
    axes[1].boxplot(
        [correct_confidences, incorrect_confidences],
        labels=["Correct", "Incorrect"],
        patch_artist=True,
        boxprops=dict(facecolor="lightblue", edgecolor="black", linewidth=2),
        medianprops=dict(color="red", linewidth=2),
    )
    axes[1].set_ylabel("Confidence", fontsize=12, fontweight="bold")
    axes[1].set_title("Confidence: Correct vs Incorrect", fontsize=13, fontweight="bold")
    axes[1].grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n✓ Confidence statistics:")
    print(f"  Correct predictions:")
    print(f"    Mean: {correct_confidences.mean():.4f}")
    print(f"    Median: {np.median(correct_confidences):.4f}")
    print(f"    Std: {correct_confidences.std():.4f}")
    print(f"\n  Incorrect predictions:")
    print(f"    Mean: {incorrect_confidences.mean():.4f}")
    print(f"    Median: {np.median(incorrect_confidences):.4f}")
    print(f"    Std: {incorrect_confidences.std():.4f}")

    # Model calibration
    print("\n" + "=" * 60)
    print("6. MODEL CALIBRATION")
    print("=" * 60)

    # Bin predictions by confidence
    bins = [0.0, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0]
    bin_names = ["0-50%", "50-60%", "60-70%", "70-80%", "80-90%", "90-95%", "95-100%"]

    print("\nCalibration: Does 80% confidence = 80% accuracy?\n")
    print(f"{'Confidence Range':<15} {'Count':<10} {'Actual Accuracy':<18} {'Gap'}")
    print("-" * 60)

    for i in range(len(bins) - 1):
        mask = (all_confidences >= bins[i]) & (all_confidences < bins[i + 1])
        count = mask.sum()
        if count > 0:
            accuracy = correct_mask[mask].mean()
            expected = (bins[i] + bins[i + 1]) / 2
            gap = accuracy - expected
            print(f"{bin_names[i]:<15} {count:<10} {accuracy*100:>15.2f}% {gap*100:>10.2f}%")

    print("\n✓ Well-calibrated model: Actual accuracy matches confidence")
    print("  Large gap = overconfident or underconfident")

    # Project summary
    print("\n" + "=" * 60)
    print("PROJECT SUMMARY")
    print("=" * 60)
    print(f"\n✓ Dataset: MNIST (handwritten digits)")
    print(f"✓ Training samples: {len(X_train)}")
    print(f"✓ Test samples: {len(X_test)}")
    print(f"✓ Classes: 10 (digits 0-9)")
    print(f"\n✓ Model Architecture:")
    print(f"  • Conv2D (32 filters, 3×3) → MaxPool")
    print(f"  • Conv2D (64 filters, 3×3) → MaxPool")
    print(f"  • Dense (128) → Dropout → Dense (10)")
    print(f"  • Total parameters: {model.count_params():,}")
    print(f"\n✓ Performance:")
    print(f"  • Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"  • Total errors: {len(misclassified_idx)}/{len(y_test)}")
    print(f"  • Average confidence (correct): {correct_confidences.mean():.4f}")
    print(f"  • Average confidence (incorrect): {incorrect_confidences.mean():.4f}")
    print("\n✓ Key insights:")
    print(
        f"  • Most confused pairs: {confusion_pairs[0][0]}→{confusion_pairs[0][1]} ({confusion_pairs[0][2]} times)"
    )
    print(
        f"  • Best recognized digit: {class_accuracies.argmax()} ({class_accuracies.max()*100:.2f}%)"
    )
    print(f"  • Hardest digit: {class_accuracies.argmin()} ({class_accuracies.min()*100:.2f}%)")
    print("=" * 60)

else:
    print("\nSkipping project (TensorFlow not available)")

## 7. Data Augmentation

**Data Augmentation**: Artificially increase dataset size by creating modified versions of images.

### Why Data Augmentation?

**Problems it solves:**
1. **Overfitting**: Model memorizes training data
2. **Small datasets**: Need more training examples
3. **Poor generalization**: Model fails on new data

**Benefits:**
- ✅ 2-10× larger effective dataset
- ✅ Better generalization
- ✅ More robust to variations
- ✅ Reduces overfitting
- ✅ No additional data collection needed!

### Common Augmentation Techniques

**1. Geometric Transformations**
- **Rotation**: Rotate image by degrees
- **Flip**: Horizontal/vertical flip
- **Zoom**: Scale image in/out
- **Shift**: Translate left/right/up/down
- **Shear**: Slant image

**2. Color Transformations**
- **Brightness**: Lighter or darker
- **Contrast**: Increase/decrease contrast
- **Saturation**: More/less colorful
- **Hue**: Shift colors

**3. Noise & Distortion**
- **Gaussian noise**: Add random noise
- **Blur**: Gaussian blur
- **Cutout**: Remove random patches
- **Mixup**: Blend two images

### Augmentation Examples

**Original → Augmented**
```
Cat image → Rotated 15° → Flipped → Zoomed → Brighter
```

### When to Use Augmentation

**Always use for:**
- Small datasets (< 10K images)
- Image classification
- Object detection

**Use carefully for:**
- Medical imaging (only realistic transforms)
- Document analysis (no rotation if orientation matters)

**Avoid for:**
- Text recognition (if augmentation distorts text)
- Tasks where transforms change meaning

### Keras ImageDataGenerator

**Training with augmentation:**
```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,          # Rotate 0-20 degrees
    width_shift_range=0.2,      # Shift left/right 20%
    height_shift_range=0.2,     # Shift up/down 20%
    horizontal_flip=True,       # Random horizontal flip
    zoom_range=0.2,             # Zoom in/out 20%
    fill_mode='nearest'         # Fill empty pixels
)

# Fit on training data
datagen.fit(X_train)

# Train with augmented data
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50
)
```

### Modern Approach: Keras Layers

**Built into model (preferred):**
```python
model = Sequential([
    # Augmentation layers (only active during training)
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    
    # Regular layers
    Conv2D(32, (3,3), activation='relu'),
    ...
])
```

**Advantages:**
- Integrated into model
- GPU accelerated
- Automatic train/test mode

### Augmentation Best Practices

**1. Start Simple**
- Begin with flip + rotation
- Add more if needed

**2. Keep Realistic**
- Don't rotate documents 90°
- Don't flip text
- Medical images: only valid transforms

**3. Monitor Performance**
- Check if augmentation helps
- Too much augmentation can hurt!

**4. Balance**
- Light augmentation: rotation ±15°, flip
- Medium: + zoom, shift
- Heavy: + color, noise, cutout

### Example Impact

| Scenario | Without Augmentation | With Augmentation |
|----------|---------------------|-------------------|
| **500 images** | 75% accuracy | 88% accuracy |
| **1,000 images** | 82% accuracy | 92% accuracy |
| **5,000 images** | 91% accuracy | 95% accuracy |

**Biggest gains on small datasets!**

Let's implement and visualize data augmentation!

In [ ]:
# Data Augmentation - Implementation and Visualization

print("=" * 60)
print("DATA AUGMENTATION TECHNIQUES")
print("=" * 60)

if has_tf:
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from scipy import ndimage

    # Select a sample image
    sample_img = X_test[0].reshape(28, 28, 1)

    print("\n" + "=" * 60)
    print("1. BASIC AUGMENTATION TECHNIQUES")
    print("=" * 60)

    fig, axes = plt.subplots(3, 4, figsize=(16, 12))

    # Original
    axes[0, 0].imshow(sample_img.reshape(28, 28), cmap="gray")
    axes[0, 0].set_title("Original", fontsize=12, fontweight="bold")
    axes[0, 0].axis("off")

    # 1. Rotation
    rotated = ndimage.rotate(sample_img.reshape(28, 28), angle=15, reshape=False)
    axes[0, 1].imshow(rotated, cmap="gray")
    axes[0, 1].set_title("Rotated (+15°)", fontsize=12, fontweight="bold")
    axes[0, 1].axis("off")

    # 2. Horizontal flip
    flipped_h = np.fliplr(sample_img.reshape(28, 28))
    axes[0, 2].imshow(flipped_h, cmap="gray")
    axes[0, 2].set_title("Horizontal Flip", fontsize=12, fontweight="bold")
    axes[0, 2].axis("off")

    # 3. Vertical flip
    flipped_v = np.flipud(sample_img.reshape(28, 28))
    axes[0, 3].imshow(flipped_v, cmap="gray")
    axes[0, 3].set_title("Vertical Flip", fontsize=12, fontweight="bold")
    axes[0, 3].axis("off")

    # 4. Zoom (simple crop and resize)
    zoom_factor = 0.8
    h, w = sample_img.shape[:2]
    h_new, w_new = int(h * zoom_factor), int(w * zoom_factor)
    start_h, start_w = (h - h_new) // 2, (w - w_new) // 2
    zoomed = sample_img[start_h : start_h + h_new, start_w : start_w + w_new]
    zoomed = tf.image.resize(zoomed, [28, 28]).numpy()
    axes[1, 0].imshow(zoomed.reshape(28, 28), cmap="gray")
    axes[1, 0].set_title("Zoom In (0.8×)", fontsize=12, fontweight="bold")
    axes[1, 0].axis("off")

    # 5. Brightness adjustment
    brighter = np.clip(sample_img * 1.5, 0, 1)
    axes[1, 1].imshow(brighter.reshape(28, 28), cmap="gray")
    axes[1, 1].set_title("Brighter (1.5×)", fontsize=12, fontweight="bold")
    axes[1, 1].axis("off")

    # 6. Darker
    darker = sample_img * 0.6
    axes[1, 2].imshow(darker.reshape(28, 28), cmap="gray")
    axes[1, 2].set_title("Darker (0.6×)", fontsize=12, fontweight="bold")
    axes[1, 2].axis("off")

    # 7. Gaussian noise
    noise = np.random.normal(0, 0.1, sample_img.shape)
    noisy = np.clip(sample_img + noise, 0, 1)
    axes[1, 3].imshow(noisy.reshape(28, 28), cmap="gray")
    axes[1, 3].set_title("+ Gaussian Noise", fontsize=12, fontweight="bold")
    axes[1, 3].axis("off")

    # 8. Shift right
    shifted = np.roll(sample_img, shift=5, axis=1)
    axes[2, 0].imshow(shifted.reshape(28, 28), cmap="gray")
    axes[2, 0].set_title("Shift Right", fontsize=12, fontweight="bold")
    axes[2, 0].axis("off")

    # 9. Shift down
    shifted_down = np.roll(sample_img, shift=5, axis=0)
    axes[2, 1].imshow(shifted_down.reshape(28, 28), cmap="gray")
    axes[2, 1].set_title("Shift Down", fontsize=12, fontweight="bold")
    axes[2, 1].axis("off")

    # 10. Combination (rotate + flip)
    combo = ndimage.rotate(flipped_h, angle=-10, reshape=False)
    axes[2, 2].imshow(combo, cmap="gray")
    axes[2, 2].set_title("Rotate + Flip", fontsize=12, fontweight="bold")
    axes[2, 2].axis("off")

    # 11. Combination (zoom + noise)
    combo2 = np.clip(zoomed + noise, 0, 1)
    axes[2, 3].imshow(combo2.reshape(28, 28), cmap="gray")
    axes[2, 3].set_title("Zoom + Noise", fontsize=12, fontweight="bold")
    axes[2, 3].axis("off")

    plt.suptitle("Data Augmentation Techniques", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    print("\n✓ Demonstrated 11 augmentation techniques")
    print("  Each creates a new training example from 1 image!")

    # ImageDataGenerator demonstration
    print("\n" + "=" * 60)
    print("2. KERAS IMAGEDATAGENERATOR")
    print("=" * 60)

    # Create augmentation generator
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,  # Don't flip digits
        fill_mode="nearest",
    )

    # Generate augmented images
    sample_batch = sample_img.reshape(1, 28, 28, 1)

    fig, axes = plt.subplots(3, 5, figsize=(18, 11))
    axes = axes.ravel()

    # Original
    axes[0].imshow(sample_img.reshape(28, 28), cmap="gray")
    axes[0].set_title("Original", fontsize=12, fontweight="bold", color="blue")
    axes[0].axis("off")

    # Generate 14 augmented versions
    aug_iter = datagen.flow(sample_batch, batch_size=1)
    for i in range(1, 15):
        aug_img = next(aug_iter)[0]
        axes[i].imshow(aug_img.reshape(28, 28), cmap="gray")
        axes[i].set_title(f"Augmented {i}", fontsize=11)
        axes[i].axis("off")

    plt.suptitle(
        "ImageDataGenerator: 14 Augmented Versions of Same Image", fontsize=14, fontweight="bold"
    )
    plt.tight_layout()
    plt.show()

    print("\n✓ Generated 14 augmented versions using ImageDataGenerator")
    print("  Settings: rotation (±20°), shift (±20%), zoom (±20%)")

    # Show effect on multiple images
    print("\n" + "=" * 60)
    print("3. AUGMENTATION ON MULTIPLE IMAGES")
    print("=" * 60)

    fig, axes = plt.subplots(5, 6, figsize=(18, 15))

    for row in range(5):
        # Original
        img_idx = row * 10
        original = X_test[img_idx].reshape(1, 28, 28, 1)
        axes[row, 0].imshow(original.reshape(28, 28), cmap="gray")
        axes[row, 0].set_title(f"Digit {y_test[img_idx]}", fontsize=11, fontweight="bold")
        axes[row, 0].axis("off")

        # 5 augmented versions
        aug_iter = datagen.flow(original, batch_size=1)
        for col in range(1, 6):
            aug_img = next(aug_iter)[0]
            axes[row, col].imshow(aug_img.reshape(28, 28), cmap="gray")
            if row == 0:
                axes[row, col].set_title(f"Aug {col}", fontsize=11)
            axes[row, col].axis("off")

    plt.suptitle("Data Augmentation: Creating Training Variations", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    print("\n✓ Applied augmentation to 5 different digits")
    print("  Each digit → 5 augmented versions")
    print("  Effective dataset size: 6× larger!")

    # Modern Keras layers approach
    print("\n" + "=" * 60)
    print("4. MODERN APPROACH: KERAS AUGMENTATION LAYERS")
    print("=" * 60)

    # Create model with built-in augmentation
    augmentation_layers = keras.Sequential(
        [
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),  # ±10%
            layers.RandomZoom(0.1),  # ±10%
        ],
        name="augmentation",
    )

    print("\n✓ Created augmentation pipeline:")
    print("  • RandomFlip (horizontal)")
    print("  • RandomRotation (±10%)")
    print("  • RandomZoom (±10%)")
    print("\n💡 These layers are built into the model")
    print("  • Only active during training")
    print("  • GPU accelerated")
    print("  • No preprocessing needed!")

    # Visualize modern augmentation
    fig, axes = plt.subplots(2, 6, figsize=(18, 6))

    test_img = X_test[5].reshape(1, 28, 28, 1)

    # Original
    axes[0, 0].imshow(test_img.reshape(28, 28), cmap="gray")
    axes[0, 0].set_title("Original", fontsize=12, fontweight="bold", color="blue")
    axes[0, 0].axis("off")

    # Apply augmentation 11 times
    for idx in range(11):
        row = idx // 6
        col = (idx + 1) % 6

        augmented = augmentation_layers(test_img, training=True)
        axes[row, col].imshow(augmented[0].numpy().reshape(28, 28), cmap="gray")
        axes[row, col].set_title(f"Aug {idx+1}", fontsize=11)
        axes[row, col].axis("off")

    plt.suptitle("Modern Keras Augmentation Layers", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.show()

    print("\n✓ Generated 11 augmented versions using Keras layers")

    # Comparison of dataset sizes
    print("\n" + "=" * 60)
    print("5. EFFECTIVE DATASET SIZE WITH AUGMENTATION")
    print("=" * 60)

    original_size = 50000  # MNIST training set
    augmentations_per_image = [0, 1, 2, 5, 10, 20]
    effective_sizes = [original_size * (1 + aug) for aug in augmentations_per_image]

    fig, ax = plt.subplots(figsize=(12, 6))

    bars = ax.bar(
        range(len(augmentations_per_image)),
        effective_sizes,
        color=["red", "orange", "yellow", "lightgreen", "green", "darkgreen"],
        edgecolor="black",
        linewidth=2,
    )

    # Add value labels
    for idx, (bar, size) in enumerate(zip(bars, effective_sizes)):
        height = bar.get_height()
        multiplier = augmentations_per_image[idx] + 1
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{size:,}\n({multiplier}× original)",
            ha="center",
            va="bottom",
            fontsize=11,
            fontweight="bold",
        )

    ax.set_xticks(range(len(augmentations_per_image)))
    ax.set_xticklabels([f"{aug} aug/img" for aug in augmentations_per_image], fontsize=11)
    ax.set_xlabel("Augmentations per Image", fontsize=13, fontweight="bold")
    ax.set_ylabel("Effective Dataset Size", fontsize=13, fontweight="bold")
    ax.set_title(
        "Data Augmentation Increases Effective Dataset Size", fontsize=14, fontweight="bold"
    )
    ax.grid(axis="y", alpha=0.3, linestyle="--")

    plt.tight_layout()
    plt.show()

    print("\n✓ Dataset size multiplication:")
    for aug, size in zip(augmentations_per_image, effective_sizes):
        multiplier = aug + 1
        print(f"  {aug} aug/image: {original_size:,} → {size:,} ({multiplier}× original)")

    print("\n" + "=" * 60)
    print("DATA AUGMENTATION SUMMARY")
    print("=" * 60)
    print("✓ Artificially increases dataset size")
    print("  • Apply random transforms to training images")
    print("  • Each epoch sees different variations")
    print("\n✓ Benefits:")
    print("  • Reduces overfitting")
    print("  • Improves generalization")
    print("  • No additional data collection!")
    print("\n✓ Implementation:")
    print("  • Classic: ImageDataGenerator")
    print("  • Modern: Keras augmentation layers (preferred)")
    print("\n✓ Best practices:")
    print("  • Use realistic transforms only")
    print("  • Start simple (flip, rotate, zoom)")
    print("  • Monitor validation performance")
    print("  • Biggest gains on small datasets")
    print("=" * 60)

else:
    print("\nSkipping data augmentation (TensorFlow not available)")

## 8. Model Deployment

**Deployment**: Making your trained model available for real-world use!

### Why Deployment Matters

**Training ≠ Production**

You've built a great model, but now what?
- How do users interact with it?
- How do you serve predictions?
- How to handle 1000s of requests?

### Deployment Options

**1. Save Model for Later Use**
```python
# Save entire model
model.save('my_model.h5')

# Load later
loaded_model = tf.keras.models.load_model('my_model.h5')

# Make predictions
predictions = loaded_model.predict(new_images)
```

**2. REST API (Flask/FastAPI)**
- Build web API
- Receive images via HTTP
- Return predictions as JSON
- Good for: Web apps, mobile apps

**3. Cloud Deployment**
- **AWS SageMaker**: Managed ML platform
- **Google Cloud AI Platform**: Auto-scaling
- **Azure ML**: Enterprise integration
- Good for: Production systems

**4. Edge Deployment**
- **TensorFlow Lite**: Mobile/IoT devices
- **ONNX**: Cross-platform format
- **TensorFlow.js**: Browser deployment
- Good for: Mobile apps, offline use

**5. Batch Processing**
- Process many images at once
- Schedule jobs (daily/weekly)
- Good for: Reports, batch analysis

### Deployment Checklist

**Before deploying:**
- ✅ Model trained and validated
- ✅ Acceptable accuracy on test set
- ✅ Error analysis completed
- ✅ Model saved in portable format
- ✅ Input preprocessing documented
- ✅ Output format defined

**Production considerations:**
- 🔍 **Monitoring**: Track prediction accuracy over time
- ⚡ **Performance**: Response time < 100ms?
- 🔒 **Security**: Authentication, rate limiting
- 📊 **Logging**: Save predictions for analysis
- 🔄 **Versioning**: Track model versions
- 🚨 **Alerts**: Notify if accuracy drops

### Simple Flask API Example

```python
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np

app = Flask(__name__)
model = tf.keras.models.load_model('digit_classifier.h5')

@app.route('/predict', methods=['POST'])
def predict():
    # Get image from request
    file = request.files['image']
    img = load_and_preprocess(file)
    
    # Make prediction
    prediction = model.predict(img)
    digit = int(np.argmax(prediction))
    confidence = float(np.max(prediction))
    
    # Return JSON
    return jsonify({
        'digit': digit,
        'confidence': confidence
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

**Usage:**
```bash
curl -X POST -F "image=@digit.png" http://localhost:5000/predict
```

### TensorFlow Lite (Mobile)

**Convert to TFLite:**
```python
# Convert model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
```

**Benefits:**
- 10-100× smaller model
- Faster inference
- Runs on mobile/embedded devices

### Model Serving Best Practices

**1. Preprocessing Pipeline**
- Document all preprocessing steps
- Package with model
- Ensure consistency

**2. Versioning**
```
models/
  ├── digit_classifier_v1.h5
  ├── digit_classifier_v2.h5
  └── digit_classifier_v3.h5  (current)
```

**3. A/B Testing**
- Deploy new model to 10% of users
- Compare with old model
- Gradually increase if better

**4. Fallback Strategy**
- If model fails, return default
- Log errors for debugging
- Don't crash application!

**5. Monitoring**
```python
# Track metrics
latency = time_end - time_start
prediction_confidence = max(predictions)

# Log
logger.info(f"Prediction: {digit}, Confidence: {confidence:.2f}, Latency: {latency:.3f}s")

# Alert if confidence low
if confidence < 0.5:
    send_alert("Low confidence prediction!")
```

### Deployment Comparison

| Method | Difficulty | Cost | Scalability | Use Case |
|--------|-----------|------|-------------|----------|
| **Local Script** | Easy | Free | Low | Development |
| **Flask API** | Medium | Low | Medium | Small apps |
| **Cloud (AWS/GCP)** | Hard | High | High | Production |
| **TFLite Mobile** | Medium | Low | N/A | Mobile apps |
| **TF.js Browser** | Medium | Low | High | Web apps |

### Common Deployment Issues

**Problem: Slow predictions**
- Solution: Use GPU, batch requests, optimize model

**Problem: Model file too large**
- Solution: Quantization, pruning, distillation

**Problem: Different results in production**
- Solution: Check preprocessing, library versions

**Problem: Out of memory**
- Solution: Reduce batch size, use smaller model

Let's save and deploy our model!

In [ ]:
# Model Deployment - Saving and Loading Models

print("=" * 60)
print("MODEL DEPLOYMENT")
print("=" * 60)

if has_tf:
    import os
    import json

    # Define save paths
    model_dir = "notebooks/outputs"
    os.makedirs(model_dir, exist_ok=True)

    model_path = os.path.join(model_dir, "digit_classifier.h5")

    print("\n" + "=" * 60)
    print("1. SAVING THE MODEL")
    print("=" * 60)

    # Save model
    model.save(model_path)

    file_size_mb = os.path.getsize(model_path) / (1024 * 1024)

    print(f"\n✓ Model saved successfully!")
    print(f"  Path: {model_path}")
    print(f"  Size: {file_size_mb:.2f} MB")
    print(f"  Format: Keras HDF5")

    # Also save as SavedModel format (TensorFlow native)
    savedmodel_dir = os.path.join(model_dir, "digit_classifier_savedmodel")
    model.save(savedmodel_dir, save_format="tf")
    print(f"\n✓ Also saved in TensorFlow SavedModel format")
    print(f"  Path: {savedmodel_dir}/")

    # Load model back
    print("\n" + "=" * 60)
    print("2. LOADING THE MODEL")
    print("=" * 60)

    loaded_model = tf.keras.models.load_model(model_path)

    print(f"\n✓ Model loaded successfully!")
    print(f"  Architecture: {len(loaded_model.layers)} layers")
    print(f"  Parameters: {loaded_model.count_params():,}")

    # Verify loaded model works
    print("\n" + "=" * 60)
    print("3. VERIFY LOADED MODEL")
    print("=" * 60)

    # Make predictions with loaded model
    test_samples = X_test[:5]

    original_preds = model.predict(test_samples, verbose=0)
    loaded_preds = loaded_model.predict(test_samples, verbose=0)

    # Check if predictions match
    predictions_match = np.allclose(original_preds, loaded_preds)

    print(f"\n✓ Verification:")
    print(f"  Predictions match: {predictions_match}")
    print(f"  Max difference: {np.max(np.abs(original_preds - loaded_preds)):.10f}")

    # Create deployment package
    print("\n" + "=" * 60)
    print("4. CREATE DEPLOYMENT PACKAGE")
    print("=" * 60)

    # Save preprocessing info
    preprocessing_info = {
        "input_shape": [28, 28, 1],
        "pixel_range": [0.0, 1.0],
        "normalization": "divide by 255",
        "color_mode": "grayscale",
        "classes": list(range(10)),
        "class_names": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    }

    info_path = os.path.join(model_dir, "model_info.json")
    with open(info_path, "w") as f:
        json.dump(preprocessing_info, f, indent=2)

    print("\n✓ Deployment package created:")
    print(f"  Model: {model_path}")
    print(f"  Info: {info_path}")
    print("\nPreprocessing requirements:")
    for key, value in preprocessing_info.items():
        print(f"  {key}: {value}")

    # Demonstrate inference function
    print("\n" + "=" * 60)
    print("5. PRODUCTION INFERENCE FUNCTION")
    print("=" * 60)

    def predict_digit(image, model_path, return_confidence=True):
        """
        Production-ready inference function.

        Args:
            image: NumPy array of shape (28, 28) or (28, 28, 1)
            model_path: Path to saved model
            return_confidence: Whether to return confidence score

        Returns:
            dict with 'digit' and optionally 'confidence'
        """
        import time

        start_time = time.time()

        # Load model (in production, load once at startup)
        model = tf.keras.models.load_model(model_path)

        # Preprocess
        if image.ndim == 2:
            image = image.reshape(1, 28, 28, 1)
        elif image.ndim == 3:
            image = image.reshape(1, 28, 28, 1)

        # Ensure float32 and normalized
        image = image.astype("float32")
        if image.max() > 1.0:
            image = image / 255.0

        # Predict
        prediction = model.predict(image, verbose=0)
        digit = int(np.argmax(prediction))
        confidence = float(np.max(prediction))

        latency = time.time() - start_time

        result = {"digit": digit, "latency_ms": round(latency * 1000, 2)}

        if return_confidence:
            result["confidence"] = round(confidence, 4)
            result["all_probabilities"] = prediction[0].tolist()

        return result

    # Test inference function
    test_image = X_test[0].reshape(28, 28)
    true_label = y_test[0]

    result = predict_digit(test_image, model_path)

    print("\n✓ Inference function test:")
    print(f"  True label: {true_label}")
    print(f"  Predicted: {result['digit']}")
    print(f"  Confidence: {result['confidence']:.2%}")
    print(f"  Latency: {result['latency_ms']:.2f} ms")
    print(f"  Correct: {'✓' if result['digit'] == true_label else '✗'}")

    # Batch prediction performance
    print("\n" + "=" * 60)
    print("6. BATCH PREDICTION PERFORMANCE")
    print("=" * 60)

    import time

    batch_sizes = [1, 10, 50, 100]

    print(f"\n{'Batch Size':<12} {'Total Time':<12} {'Time/Image':<15} {'Throughput'}")
    print("-" * 60)

    for batch_size in batch_sizes:
        batch = X_test[:batch_size]

        start = time.time()
        preds = loaded_model.predict(batch, verbose=0)
        elapsed = time.time() - start

        time_per_image = (elapsed / batch_size) * 1000  # ms
        throughput = batch_size / elapsed  # images/sec

        print(
            f"{batch_size:<12} {elapsed:>10.3f}s {time_per_image:>12.2f} ms {throughput:>12.1f} img/s"
        )

    print("\n✓ Batch processing is more efficient!")
    print("  Larger batches = better GPU utilization")

    # Model conversion demonstration
    print("\n" + "=" * 60)
    print("7. MODEL CONVERSION (TensorFlow Lite)")
    print("=" * 60)

    try:
        # Convert to TFLite
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()

        # Save TFLite model
        tflite_path = os.path.join(model_dir, "digit_classifier.tflite")
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)

        # Compare sizes
        h5_size = os.path.getsize(model_path) / 1024  # KB
        tflite_size = os.path.getsize(tflite_path) / 1024  # KB
        compression_ratio = h5_size / tflite_size

        print("\n✓ TensorFlow Lite model created!")
        print(f"  Original (H5): {h5_size:.1f} KB")
        print(f"  TFLite: {tflite_size:.1f} KB")
        print(f"  Compression: {compression_ratio:.1f}× smaller!")
        print(f"\n💡 TFLite benefits:")
        print("  • Optimized for mobile/embedded devices")
        print("  • Faster inference")
        print("  • Smaller file size")

    except Exception as e:
        print(f"\n⚠ TFLite conversion failed: {e}")
        print("  (This is normal in some environments)")

    # Deployment summary
    print("\n" + "=" * 60)
    print("DEPLOYMENT SUMMARY")
    print("=" * 60)

    print("\n✓ Model ready for deployment!")
    print(f"\nFiles created:")
    print(f"  1. {model_path}")
    print(f"  2. {savedmodel_dir}/")
    print(f"  3. {info_path}")
    if os.path.exists(os.path.join(model_dir, "digit_classifier.tflite")):
        print(f"  4. {os.path.join(model_dir, 'digit_classifier.tflite')}")

    print("\n✓ Next steps for deployment:")
    print("  1. Choose deployment method (API, cloud, mobile)")
    print("  2. Set up serving infrastructure")
    print("  3. Implement monitoring and logging")
    print("  4. Test with real data")
    print("  5. Deploy to production!")

    print("\n✓ Example deployment commands:")
    print("\n  Flask API:")
    print("    python app.py")
    print("\n  Docker:")
    print("    docker build -t digit-classifier .")
    print("    docker run -p 5000:5000 digit-classifier")
    print("\n  Cloud (AWS):")
    print("    aws sagemaker create-model ...")

    print("\n" + "=" * 60)
    print("💡 DEPLOYMENT BEST PRACTICES")
    print("=" * 60)
    print("\n1. Versioning:")
    print("  • Use semantic versioning (v1.0.0, v1.1.0, v2.0.0)")
    print("  • Keep old versions for rollback")
    print("\n2. Monitoring:")
    print("  • Track prediction latency")
    print("  • Monitor confidence scores")
    print("  • Log errors and edge cases")
    print("\n3. Security:")
    print("  • Add authentication (API keys)")
    print("  • Rate limiting (prevent abuse)")
    print("  • Input validation (check image format)")
    print("\n4. Performance:")
    print("  • Use batching for multiple requests")
    print("  • Cache frequent predictions")
    print("  • Use GPU for high throughput")
    print("\n5. Testing:")
    print("  • A/B test new models")
    print("  • Canary deployments (gradual rollout)")
    print("  • Monitor performance metrics")
    print("=" * 60)

else:
    print("\nSkipping deployment (TensorFlow not available)")

## 9. Exercises

Apply your Computer Vision and CNN knowledge with these hands-on exercises!

### Exercise 1: Build a Simple CNN (Easy)
**Task**: Create a CNN for MNIST with the following architecture:
- Conv2D (16 filters, 3×3) → ReLU → MaxPool
- Conv2D (32 filters, 3×3) → ReLU → MaxPool
- Dense (64) → Dropout (0.3) → Dense (10)

**Goals**:
- Achieve >95% test accuracy
- Count total parameters
- Compare with a fully connected network

**Hints**:
- Use padding='same' to preserve dimensions
- Train for 10 epochs
- Use Adam optimizer

---

### Exercise 2: Data Augmentation Experiment (Medium)
**Task**: Train two models on a subset (1000 images) of MNIST:
1. Without augmentation
2. With augmentation (rotation, zoom, shift)

**Goals**:
- Compare test accuracies
- Plot training curves
- Analyze which benefits more from augmentation

**Hints**:
- Use ImageDataGenerator or Keras augmentation layers
- Train both for same number of epochs
- Use same architecture for fair comparison

---

### Exercise 3: Transfer Learning (Medium)
**Task**: Use MobileNetV2 for a custom classification task:
- Load pre-trained MobileNetV2 (ImageNet weights)
- Freeze base layers
- Add custom classifier
- Train on CIFAR-10 (or subset of MNIST)

**Goals**:
- Achieve better accuracy than training from scratch
- Compare training time
- Visualize learned features

**Hints**:
- Resize images to 32×32 minimum
- Convert grayscale to RGB if needed
- Use small learning rate

---

### Exercise 4: Error Analysis (Medium)
**Task**: Analyze misclassifications from your trained model:
- Find all misclassified examples
- Group by confusion pairs (e.g., 3→8, 5→3)
- Visualize most common errors
- Propose improvements

**Goals**:
- Understand model weaknesses
- Identify systematic errors
- Create error report

**Hints**:
- Use confusion matrix
- Sort by frequency
- Look for patterns in errors

---

### Exercise 5: Custom Dataset (Hard)
**Task**: Create a CNN for a custom image classification task:
- Collect/download a small dataset (100+ images, 3+ classes)
- Preprocess images (resize, normalize)
- Build and train CNN
- Evaluate and deploy

**Suggestions**:
- Dog breeds (3-5 breeds)
- Fruits (apple, banana, orange)
- Hand gestures (rock, paper, scissors)
- Traffic signs

**Goals**:
- Complete end-to-end pipeline
- Handle real-world data
- Deploy working model

**Hints**:
- Start with small dataset
- Use data augmentation heavily
- Consider transfer learning
- Save model for reuse

---

### Exercise 6: Model Optimization (Hard)
**Task**: Optimize your CNN for production:
- Reduce model size by 50%+
- Maintain >90% of original accuracy
- Measure inference speed

**Techniques to try**:
- Reduce filters per layer
- Use fewer layers
- Pruning
- Quantization (TFLite)
- Knowledge distillation

**Goals**:
- Smaller model file
- Faster inference
- Minimal accuracy loss

---

### Exercise 7: Real-time Deployment (Advanced)
**Task**: Deploy your digit classifier as a web app:
- Create Flask/FastAPI endpoint
- Accept image uploads
- Return predictions
- Add visualization

**Features**:
- Upload image (PNG/JPG)
- Display predicted digit
- Show confidence score
- Visualize top-3 predictions

**Goals**:
- Working web service
- Handle edge cases
- Production-ready code

**Hints**:
- Use Flask for simplicity
- Handle image formats correctly
- Add error handling
- Test with various inputs

---

### Bonus Challenges

**Challenge 1**: Build VGG-style network
- Stack multiple 3×3 convolutions
- Compare with simpler CNN

**Challenge 2**: Implement Grad-CAM
- Visualize which parts of image the CNN focuses on
- Interpret model decisions

**Challenge 3**: Multi-task Learning
- Predict both digit class AND whether it's even/odd
- Single model, two outputs

**Challenge 4**: Ensemble Methods
- Train 5 different CNNs
- Combine predictions (voting or averaging)
- Beat single model performance

---

### Exercise Solutions

Solutions for these exercises can be found in:
- `notebooks/solutions/17_cnn_exercises_solutions.ipynb`

**Remember**: Try exercises yourself first before checking solutions!

In [ ]:
# Exercise Workspace

# Choose an exercise and implement it here!

# Exercise 1: Build a Simple CNN
# ---------------------------------
# YOUR CODE HERE


# Exercise 2: Data Augmentation Experiment
# -----------------------------------------
# YOUR CODE HERE


# Exercise 3: Transfer Learning
# ------------------------------
# YOUR CODE HERE


# Exercise 4: Error Analysis
# ---------------------------
# YOUR CODE HERE


# Exercise 5: Custom Dataset
# ---------------------------
# YOUR CODE HERE


# Exercise 6: Model Optimization
# -------------------------------
# YOUR CODE HERE


# Exercise 7: Real-time Deployment
# ---------------------------------
# YOUR CODE HERE

## 10. Key Takeaways

Congratulations on completing Module 17: Computer Vision with CNNs!

### Core Concepts Mastered

**1. Computer Vision Fundamentals**
- Images are matrices of pixel values (0-255)
- RGB images have 3 channels (Red, Green, Blue)
- CNNs automatically learn hierarchical features
- Traditional ML requires hand-crafted features

**2. Convolutional Neural Networks (CNNs)**
- Convolution operation: slide filters across image
- Each filter detects different patterns (edges, textures, objects)
- Parameters = (filter_size² × input_channels + 1) × num_filters
- Stride controls how much filter moves
- Padding preserves spatial dimensions

**3. Pooling and Padding**
- Max pooling: downsample by taking maximum in each region
- Reduces computational cost and overfitting
- Padding adds zeros around border to preserve dimensions
- Typical pattern: CONV → ReLU → POOL

**4. CNN Architecture**
- Standard: INPUT → [CONV → ReLU → POOL]×N → FC → OUTPUT
- Most parameters in Dense layers, not Conv layers
- Receptive field grows with depth
- Famous architectures: LeNet, AlexNet, VGG, ResNet

**5. Transfer Learning**
- Use pre-trained models (ImageNet weights)
- Freeze base model, train only classifier
- 10-100× faster training, better accuracy on small datasets
- Fine-tuning: unfreeze last layers with small learning rate

**6. Data Augmentation**
- Artificially increase dataset size (2-10×)
- Techniques: rotation, flip, zoom, shift, brightness
- Biggest gains on small datasets
- Modern approach: Keras augmentation layers

**7. Model Evaluation**
- Confusion matrix shows class-wise errors
- Classification report: precision, recall, F1-score
- Error analysis: understand systematic failures
- Confidence calibration: does 80% confidence = 80% accuracy?

**8. Model Deployment**
- Save model: .h5 (Keras), SavedModel (TensorFlow)
- Deployment options: Flask API, cloud (AWS/GCP), mobile (TFLite)
- Production considerations: monitoring, versioning, security
- Batch processing more efficient than single predictions

### Key Formulas

**Output Size After Convolution:**
```
Output = (Input - Filter + 2×Padding) / Stride + 1
```

**Example**: 28×28 input, 3×3 filter, stride=1, padding=0
- Output = (28 - 3 + 0) / 1 + 1 = 26×26

**Conv Layer Parameters:**
```
Params = (filter_h × filter_w × in_channels + 1) × num_filters
```

**Example**: 32 filters of 3×3, input has 1 channel
- Params = (3 × 3 × 1 + 1) × 32 = 320

### Performance Tips

**For Better Accuracy:**
- Use data augmentation (rotation, zoom, flip)
- Try transfer learning (pre-trained models)
- Add more Conv layers (go deeper)
- Increase filters (32 → 64 → 128)
- Use batch normalization
- Tune learning rate

**For Faster Training:**
- Use transfer learning (freeze base model)
- Smaller batch size if memory limited
- Use GPU if available
- Reduce image resolution
- Fewer filters per layer

**For Smaller Models:**
- Reduce number of filters
- Use global average pooling instead of FC layers
- Convert to TFLite (quantization)
- Knowledge distillation

### Common Mistakes to Avoid

❌ **Too many parameters in Dense layers**
→ Add more CONV/POOL layers, use Global Average Pooling

❌ **No data augmentation on small datasets**
→ Always augment when < 10K images

❌ **Training from scratch with small dataset**
→ Use transfer learning instead

❌ **Wrong input preprocessing**
→ Normalize pixels to [0,1], use correct preprocessing for pre-trained models

❌ **Not monitoring validation loss**
→ Track both train and validation metrics, stop if overfitting

❌ **Ignoring class imbalance**
→ Use class weights, balanced sampling, or focal loss

### Real-World Applications

**Medical Imaging:**
- Detect tumors in X-rays/CT scans
- Classify skin lesions (melanoma detection)
- Diagnose diabetic retinopathy

**Autonomous Vehicles:**
- Detect pedestrians, traffic signs, vehicles
- Lane detection
- Semantic segmentation of road scenes

**Security & Surveillance:**
- Face recognition
- Anomaly detection
- License plate reading

**Agriculture:**
- Crop disease detection
- Yield estimation from drone imagery
- Weed detection

**E-commerce:**
- Visual search
- Product recommendations
- Quality control

### Tools & Libraries Learned

- **TensorFlow/Keras**: Model building and training
- **sklearn**: Data splitting, metrics, evaluation
- **NumPy**: Array operations, manual convolution
- **Matplotlib/Seaborn**: Visualization
- **ImageDataGenerator**: Data augmentation
- **TFLite**: Mobile deployment

### Next Steps

**To deepen your Computer Vision skills:**

1. **Advanced Architectures:**
   - ResNet (residual connections)
   - Inception (multi-scale features)
   - EfficientNet (compound scaling)
   - Vision Transformers (ViT)

2. **Object Detection:**
   - YOLO, Faster R-CNN
   - Bounding box prediction
   - Non-max suppression

3. **Semantic Segmentation:**
   - U-Net, Mask R-CNN
   - Pixel-wise classification
   - Medical image segmentation

4. **Generative Models:**
   - GANs (Generative Adversarial Networks)
   - Stable Diffusion
   - Image-to-image translation

5. **Advanced Topics:**
   - Self-supervised learning
   - Few-shot learning
   - Neural architecture search

### Recommended Resources

**Books:**
- "Deep Learning for Computer Vision" by Rajalingappaa Shanmugamani
- "Hands-On Computer Vision with TensorFlow 2" by Planche & Andres

**Courses:**
- Stanford CS231n: Convolutional Neural Networks
- Fast.ai: Practical Deep Learning for Coders

**Datasets to Practice:**
- CIFAR-10/100 (60K images, 10/100 classes)
- ImageNet (1.2M images, 1000 classes)
- COCO (object detection, segmentation)
- Kaggle competitions (real-world challenges)

**Communities:**
- r/computervision (Reddit)
- Papers with Code (latest research)
- Kaggle forums (practical tips)

---

## Module Complete!

You've learned:
✅ Computer vision fundamentals
✅ How CNNs work (convolution, pooling, padding)
✅ Building CNN architectures
✅ Transfer learning with pre-trained models
✅ Data augmentation techniques
✅ Complete classification pipeline
✅ Model deployment strategies

**Next Module**: `18_nlp.ipynb` - Natural Language Processing

---

### Final Project Idea

**Build an Image Classifier for Your Own Domain!**

Choose a topic you're interested in:
- Medical images (COVID-19 X-rays)
- Wildlife (animal species)
- Fashion (clothing classification)
- Food (recipe suggestions)
- Art (style classification)

**Steps:**
1. Collect/download dataset (100+ images per class)
2. Preprocess and augment data
3. Try both custom CNN and transfer learning
4. Evaluate and analyze errors
5. Deploy as web app
6. Share your results!

Keep practicing and building! Computer vision is one of the most impactful areas of AI. 🚀